# versions des bibliothèques

In [84]:
import os

import sys
print("Python version: {}". format(sys.version))

import IPython
from IPython import display
from IPython.display import Markdown, display, HTML
print("IPython version: {}". format(IPython.__version__))

import numpy as np
print("NumPy version: {}". format(np.__version__))

import scipy as sp
from scipy import stats
print("SciPy version: {}". format(sp.__version__))

import pandas as pd
print("pandas version: {}". format(pd.__version__))

import matplotlib
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import seaborn as sns
print("seaborn version : {}". format(sns.__version__))
sns.set()

import tensorflow as tf
print("tensorflow version : {}". format(tf.__version__))

print("keras version : {}". format(tf.keras.__version__))

import logging

Python version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
IPython version: 5.5.0
NumPy version: 1.21.6
SciPy version: 1.4.1
pandas version: 1.3.5
matplotlib version: 3.2.2
seaborn version : 0.11.2
tensorflow version : 2.8.0
keras version : 2.8.0


# définition du niveau de trace

In [85]:
import doctest

Doc_Fonction = False
     
# Définition  du niveau de traces : DEBUG, INFO, WARNING, ERROR ou CRITICAL
     
niveauTrace = logging.DEBUG
#niveauTrace = logging.INFO
#niveauTrace = logging.WARNING
#niveauTrace = logging.ERROR
#niveauTrace = logging.CRITICAL
    
logging.basicConfig(format='%(levelname)s:%(message)s', level=niveauTrace)
    
if (Doc_Fonction):
    #"""
    # Mode d'emploi du Décorateur Trace_appel
    print("\n\n    Mode d'emploi des fonctions")
    print(    "    ---------------------------")
    help(Trace_appel)
    #"""
    
# Test unitaire des fonctions
print("\n +--------------------+")
print(  " |  Tests Unitaires   |")
print(  " +--------------------+")
doctest.testmod(verbose = True)
print("")
#"""


 +--------------------+
 |  Tests Unitaires   |
 +--------------------+
63 items had no tests:
    __main__
    __main__.AuxiliaryHead
    __main__.AuxiliaryHead.__init__
    __main__.AuxiliaryHead.call
    __main__.ChosenInputs
    __main__.ChosenInputs.__call__
    __main__.ChosenInputs.__init__
    __main__.ChosenInputs._tensor_reduction
    __main__.ContextStack
    __main__.ContextStack.__enter__
    __main__.ContextStack.__exit__
    __main__.ContextStack.__init__
    __main__.ContextStack.pop
    __main__.ContextStack.push
    __main__.ContextStack.top
    __main__.DilConv
    __main__.DilConv.__init__
    __main__.DilConv.call
    __main__.DropPath
    __main__.DropPath.__init__
    __main__.DropPath.call
    __main__.FactorizedReduce
    __main__.FactorizedReduce.__init__
    __main__.FactorizedReduce.call
    __main__.InputChoice
    __main__.InputChoice.__call__
    __main__.InputChoice.__init__
    __main__.InputChoice.__repr__
    __main__.InputChoice._key
    __main__.In

# décorateur Trace_appel

In [86]:
import cProfile
import io
import pstats
#from pstats import SortKey
import tracemalloc
from functools import wraps


def Trace_appel(Doc_Fonction=False, Mesure_Temps=False, Mesure_Memoire=False, Valeur_Arguments=True):
    """
    Décorateur qui permet de tracer l'appel des fonctions (niveau INFO)
    et la documentation de la fonction, ses arguments d'entree et  
    la valeur de retour (niveau DEBUG) sans surcharger le code
    En mode DEBUG, on peut aussi tracer le temps d'exécution en utilisant cprofile
    - ncalls = nombre d'appels
    - tottime = temps passé dans la fonction en excluant le temps passé dans les sous-fonctions appelées
    - percall = tottime / ncalls
    - cumtime = temps cumulé passé dans la fonction et dans les sous-fonctions appelées
    - percall = cumtime / ncalls
    
    Utilisation :
    @Trace_appel(information_a_tracer=True)
    def essai(x,y,z):
        code de la fonction essai
        return codeRetour
        
    """
    def Executer_appel(fonction):
        
        @wraps(fonction)
        def wrapper(*args, **kwargs):
            logging.info("")
            logging.info(">>>>>> Appel de {} >>>>>>".format(fonction.__name__))
            if (Doc_Fonction): logging.debug(fonction.__doc__)
            arguments = "  ... Arguments :"        
            for indice,valeur in enumerate(args):
                arguments += "  N°{}={} ".format(indice+1, valeur)
            for clef, valeur in kwargs.items():
                arguments += "  {}={}".format(clef, valeur)
            if (Valeur_Arguments) : logging.debug("{}".format(arguments))
            
            if (Mesure_Temps):
                # Temps d'execution
                pr = cProfile.Profile()
                pr.enable()
            
            if (Mesure_Memoire) : tracemalloc.start()
            
            retour = fonction(*args, **kwargs)
            
            if (Mesure_Temps) :
                pr.disable()
                s = io.StringIO()
                #sortby = SortKey.CUMULATIVE  # 'cumulative'
                sortby = "cumulative"  # 'cumulative'
                ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
                ps.print_stats(10)
            
            if (Valeur_Arguments) : logging.debug("  ... Resultat : {}".format(retour))
            
            if (Mesure_Temps) : 
                logging.debug("")
                logging.debug("    >>>>> Temps d'exécution >>>>")
                logging.debug(s.getvalue())
            
            if (Mesure_Memoire) :
                current, peak =  tracemalloc.get_traced_memory()
                memoire_en_octets = peak * 1.024
                if (memoire_en_octets < 1000) :
                    Unite = "octets"
                else :
                    memoire_en_octets /= 1000
                    if (memoire_en_octets < 1000) :
                        Unite = "Ko"
                    else:
                        memoire_en_octets /= 1000
                        if (memoire_en_octets < 1000) :
                            Unite = "Mo"
                        else:
                            Unite = "Go"
                      
                #if(peak )
                tracemalloc.stop()
                logging.debug("")
                logging.debug("    >>>>> Mémoire allouée : {:.3f} {} >>>>".format(memoire_en_octets,
                                                                              Unite))
            
            logging.info("----- Fin de {} -----".format(fonction.__name__))
        
            return retour
        
        return wrapper
    
    return Executer_appel

In [87]:
# Accés au github
from getpass import getpass
import os

git_public = True

if git_public:
    !git clone https://github.com/microsoft/nni
else:
    user = getpass('NAS user')
    password = getpass('NAS password')
    os.environ['NAS_AUTH'] = user + ':' + password
    !git clone https://$NAS_AUTH@github.com/microsoft/nni

Cloning into 'nni'...
remote: Enumerating objects: 53173, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 53173 (delta 0), reused 1 (delta 0), pack-reused 53168
Receiving objects: 100% (53173/53173), 117.99 MiB | 25.56 MiB/s, done.
Resolving deltas: 100% (31185/31185), done.


In [88]:
# On se positionne dans le bon réperoire
%cd nni/examples/nas/oneshot/darts/

/content/nni/examples/nas/oneshot/darts/nni/examples/nas/oneshot/darts/nni/examples/nas/oneshot/darts


In [89]:
%ls

datasets.py  ops.py     README_zh_CN.md  search.py
model.py     README.md  retrain.py       utils.py


In [90]:
!pip install nni
!pip install graphviz

In [91]:
!pip install pytorch_lightning

In [92]:
"""
%%shell
python search.py --layer=2 --epoch=2 --visualization
"""

'\n%%shell\npython search.py --layer=2 --epoch=2 --visualization\n'

Final_architecture = {'reduce_n2_p0': 'maxpool',
                      'reduce_n2_p1': 'maxpool',
                      'reduce_n3_p0': 'maxpool',
                      'reduce_n3_p1': 'maxpool',
                      'reduce_n3_p2': 'maxpool',
                      'reduce_n4_p0': 'maxpool',
                      'reduce_n4_p1': 'maxpool',
                      'reduce_n4_p2': 'maxpool',
                      'reduce_n4_p3': 'maxpool',
                      'reduce_n5_p0': 'maxpool',
                      'reduce_n5_p1': 'maxpool',
                      'reduce_n5_p2': 'maxpool',
                      'reduce_n5_p3': 'maxpool', 
                      'reduce_n5_p4': 'maxpool',
                      'reduce_n2_switch': [1, 0],
                      'reduce_n3_switch': [2, 1],
                      'reduce_n4_switch': [3, 2],
                      'reduce_n5_switch': [4, 3]}

# convertir un modèle pytorch en un modèle tensorflow

## installation des dépendances

In [93]:
!pip install torchvision

In [94]:
!pip install onnx

In [95]:
!pip install tensorflow

In [96]:
!pip install onnx_tf

## code trouvé permettant d'effectuer la conversion d'un modèle pytorch en un modèle tensorflow

La classe correspond à la définition du modèle ainsi que la façon dont il est implémenté 

In [97]:
"""
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import onnx
from collections import OrderedDict
import tensorflow as tf
from torch.autograd import Variable
from onnx_tf.backend import prepare

class MLP(nn.Module):
    def __init__(self, input_dims, n_hiddens, n_class):
        super(MLP, self).__init__()
        assert isinstance(input_dims, int), 'Please provide int for input_dims'
        self.input_dims = input_dims
        current_dims = input_dims
        layers = OrderedDict()

        if isinstance(n_hiddens, int):
            n_hiddens = [n_hiddens]
        else:
            n_hiddens = list(n_hiddens)
        for i, n_hidden in enumerate(n_hiddens):
            layers['fc{}'.format(i+1)] = nn.Linear(current_dims, n_hidden)
            layers['relu{}'.format(i+1)] = nn.ReLU()
            layers['drop{}'.format(i+1)] = nn.Dropout(0.2)
            current_dims = n_hidden
        layers['out'] = nn.Linear(current_dims, n_class)

        self.model= nn.Sequential(layers)
        print(self.model)

    def forward(self, input):
        input = input.view(input.size(0), -1)
        assert input.size(1) == self.input_dims
        return self.model.forward(input)

print("%s" % sys.argv[1])
print("%s" % sys.argv[2])


# Load the trained model from file
trained_dict = torch.load(sys.argv[1], map_location={'cuda:0': 'cpu'})

trained_model = MLP(784, [256, 256], 10)
trained_model.load_state_dict(trained_dict)

if not os.path.exists("%s" % sys.argv[2]):
    os.makedirs("%s" % sys.argv[2])

# Export the trained model to ONNX
dummy_input = Variable(torch.randn(1, 1, 28, 28)) # one black and white 28 x 28 picture will be the input to the model
torch.onnx.export(trained_model, dummy_input, "%s/mnist.onnx" % sys.argv[2])

# Load the ONNX file
model = onnx.load("%s/mnist.onnx" % sys.argv[2])

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

tf_rep.export_graph("%s/mnist.pb" % sys.argv[2])

converter = tf.lite.TFLiteConverter.from_frozen_graph(
        "%s/mnist.pb" % sys.argv[2], tf_rep.inputs, tf_rep.outputs)
tflite_model = converter.convert()
open("%s/mnist.tflite" % sys.argv[2], "wb").write(tflite_model)
"""


'\nimport sys\nimport os\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport onnx\nfrom collections import OrderedDict\nimport tensorflow as tf\nfrom torch.autograd import Variable\nfrom onnx_tf.backend import prepare\n\nclass MLP(nn.Module):\n    def __init__(self, input_dims, n_hiddens, n_class):\n        super(MLP, self).__init__()\n        assert isinstance(input_dims, int), \'Please provide int for input_dims\'\n        self.input_dims = input_dims\n        current_dims = input_dims\n        layers = OrderedDict()\n\n        if isinstance(n_hiddens, int):\n            n_hiddens = [n_hiddens]\n        else:\n            n_hiddens = list(n_hiddens)\n        for i, n_hidden in enumerate(n_hiddens):\n            layers[\'fc{}\'.format(i+1)] = nn.Linear(current_dims, n_hidden)\n            layers[\'relu{}\'.format(i+1)] = nn.ReLU()\n            layers[\'drop{}\'.format(i+1)] = nn.Dropout(0.2)\n            current_dims = n_hidden\n        layers[\'out\'] = nn.L

pb: ce code ne semble marcher qu'en local

## utilisation de la librairie pytorch2keras 

In [98]:
!pip install pytorch2keras 

In [99]:
#!pip install onnx

## transformation des opérations du modèle pytorch en modèle tensorflow à la main

### Code tensorflow de la classe DilConv

In [100]:
import tensorflow as tf
from tensorflow.keras import Sequential

In [101]:
class DilConv(tf.keras.layers.Layer):
  """
  This class is the dilated depthwise separable convolution class.
  """
  def __init__(self, C_in, C_out, kernel_size, stride, padding, dilation, affine=True):
    """
    Description
    ---------------
    This function initializes an instance of the DilConv class.
    Input(s)
    ---------------
    C_in: int
    C_out: int
    kernel_size: int
    stride: int
    padding: int
    dilation: int
    affine: boolean
    Output(s)
    ---------------
    No output
    """
    super().__init__()
    self.net = tf.keras.Sequential()
    self.net.add(tf.keras.layers.ReLU())
    self.net.add(tf.keras.layers.Conv2D(C_in, kernel_size, strides=stride, dilation_rate=dilation, padding="valid", groups=C_in, data_format="channels_last" ) )
    self.net.add(tf.keras.layers.Conv2D(C_out, kernel_size, strides=(1, 1), padding='valid', data_format="channels_last"))
    self.net.add(tf.keras.layers.BatchNormalization() )

  def call(self, x):
    """
    Description
    ---------------
    This function applies the instance of the DilConv class to the data.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    output: tf.Tensor
    """
    return self.net(x)


### Code tensorflow de la classe SepConv

In [102]:
class SepConv(tf.keras.layers.Layer):
  """
  Depthwise separable conv.
  DilConv(dilation=1) * 2.
  """
  def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
    """
    Description
    ---------------
    This function initializes an instance of the SepConv class.
    Input(s)
    ---------------
    C_in: int
    C_out: int
    kernel_size: int
    stride: int
    padding: int
    affine: boolean
    Output(s)
    ---------------
    No output
    """
    super().__init__()
    self.net = tf.keras.Sequential()
    self.net.add(DilConv(C_in, C_in, kernel_size, stride, padding, dilation=1,affine=affine))
    self.net.add(DilConv(C_in, C_out, kernel_size, 1, padding, dilation=1, affine=affine))

  def call(self, x):
    """
    Description
    ---------------
    This function applies the instance of the SepConv class to the data.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    output: tf.Tensor
    """
    return self.net(x)

In [103]:
layer = tf.keras.layers.ReLU()
output = layer([-3.0, -1.0, 0.0, 2.0])
output
list(output.numpy())

[0.0, 0.0, 0.0, 2.0]

### Code tensorflow de la classe FactorizedReduce

In [104]:
class FactorizedReduce(tf.keras.layers.Layer):

  """

  Reduce feature map size by factorized pointwise (stride=2).

  """

  def __init__(self, C_in, C_out, affine=True):
    """
    Description
    ---------------
    This function initializes an instance of the FactorizedReduce class.
    Input(s)
    ---------------
    C_in: int
    C_out: int
    Output(s)
    ---------------
    No output
    """
    print("---------------entrée dans la fonction __init__ de la classe FactorizedReduce -------------------")
    super().__init__()
    self.relu = tf.keras.layers.ReLU()
    self.conv2d_1 = tf.keras.layers.Conv2D(filters=C_out // 2,
                                      strides=2,
                                      padding='same' ,
                                      data_format="channels_last",
                                      kernel_size=C_in,
                                      use_bias=False)
    self.conv2d_2 = tf.keras.layers.Conv2D(filters=C_out // 2,
                                      strides=2,
                                      padding='same' ,
                                      data_format="channels_last",
                                      kernel_size=C_in,
                                      use_bias=False)
    self.concatenation = tf.keras.layers.Concatenate(axis=-1)
    self.batch_normalisation = tf.keras.layers.BatchNormalization(axis=-1)

  def call(self, x):
    """
    Description
    ---------------
    This function applies the instance of the FactorizedReduce class to the data.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    output: tf.Tensor
    """
    print("--------------entrée dans la fonction call de la classe FactorizedReduce -------------------")
    print("x = {}".format(x))
    if isinstance(x,tf.Tensor):
      # x is a Tensor
      print("x.get_shape() = {}".format(x.get_shape()))
      x2 = tf.keras.layers.ReLU()(x)
      print("x2 = {}".format(x2))
      out_1 = self.conv2d_1(x2)
      print("out_1 = {}".format(out_1))
      out_2 = self.conv2d_2(x2[:,:,1:,1:])
      print("out_2 = {}".format(out_2))
      out_3 = self.concatenation([out_1,out_2])
      print("out_3 = {}".format(out_3))
      out = self.batch_normalisation(out_3)
    else:
      # x is a list
      out = ()
      print("x[0] = {}".format(x[0]))
      print("len(x) = {}".format(len(x)))
      for i in range(len(x)):
        print("i = {}".format(i))
        print("len(x[i]) = {}".format(len(x[i])))
        if isinstance(len(x[i]),int):
          # we want to know if x[i] is a tensor list
          if len(x[i]) == 0 :
            # x[i] contains no element
            print("x[i] est vide")
            out = ()
          elif len(x[i]) == 1 :
            # x[i] contains only one tensor
            entree = x[i][0]
            print("entree.get_shape() = {}".format(entree.get_shape()))
            x2_i = tf.keras.layers.ReLU()(entree)
            print("x2_i = {}".format(x2_i))
            print("x2_i.get_shape() = {}".format(x2_i.get_shape()))
            print("x2_i.get_shape()[1] = {}".format(x2_i.get_shape()[1]))
            out_1_i = self.conv2d_1(x2_i)
            print("out_1_i.get_shape() = {}".format(out_1_i.get_shape()))
            out_2_i = self.conv2d_2(x2_i[:,:,1:,1:])
            print("out_2_i.get_shape() = {}".format(out_2_i))
            out_3_i = self.concatenation([out_1_i,out_2_i])
            print("out_3_i.get_shape() = {}".format(out_3_i))
            out_i = self.batch_normalisation(out_3_i)
            print("out_i.get_shape() = {}".format(out_i.get_shape()))
            out = out + (out_i,)
          else:
            # x[i] is a list containing more than one tensor
            for j in range(len(x[i])):
              entree = x[i][j]
              print("entree = {}".format(entree))
              print("entree.get_shape() = {}".format(entree.get_shape()))
              x2_i_j = tf.keras.layers.ReLU()(entree)
              print("x2_i_j = {}".format(x2_i_j))
              print("x2_i_j.get_shape() = {}".format(x2_i_j.get_shape()))
              print("x2_i_j.get_shape()[1] = {}".format(x2_i_j.get_shape()[1]))
              out_1_i_j = self.conv2d_1(x2_i_j)
              print("out_1_i_j.get_shape() = {}".format(out_1_i_j.get_shape()))
              out_2_i_j = self.conv2d_2(x2_i_j[:,:,1:,1:])
              print("out_2_i_j.get_shape() = {}".format(out_2_i_j))
              out_3_i_j = self.concatenation([out_1_i_j,out_2_i_j])
              print("out_3_i_j.get_shape() = {}".format(out_3_i_j))
              out_i_j = self.batch_normalisation(out_3_i_j)
              print("out_i_j = {}".format(out_i_j))
              print("out_i_j.get_shape() = {}".format(out_i_j.get_shape()))
              out = out + (out_i_j,)
            
        else:
          # x[i] is a Tensor
          x2_i = tf.keras.layers.ReLU()(x[i])
          print("x2_i = {}".format(x2_i))
          print("x2_i.get_shape() = {}".format(x2_i.get_shape()))
          out_1_i = self.conv2d_1(x2_i)
          print("out_1_i.get_shape() = {}".format(out_1_i.get_shape()))
          out_2_i = self.conv2d_2(x2_i[:,:,1:,1:])
          print("out_2_i.get_shape() = {}".format(out_2_i))
          out_3_i = self.concatenation([out_1_i,out_2_i])
          print("out_3_i.get_shape() = {}".format(out_3_i))
          out_i = self.batch_normalisation(out_3_i)
          print("out_i = {}".format(out_i))
          print("out_i.get_shape() = {}".format(out_i.get_shape()))
          out = out + (out_i,)
    print("out = {}".format(out))
    return out

### Code tensorflow de la classe DropPath

In [105]:
import tensorflow as tf
import tensorflow_probability as tfp

class DropPath(tf.keras.layers.Layer):
  def __init__(self, p=0):
    """
    Description
    ---------------
    Creates an instance of the class DropPath.

    Input(s)
    ---------------
    p: float (0 if you don't want, else, between 0 and 1)

    Output(s)
    ---------------
    No output
    """
    print("--------------------entrée dans la fonction ___init__ de la classe DropPath---------------------")
    super().__init__()
    print("p = {}".format(p))
    self.p=p
    print("self.p = {}".format(self.p))
  
  def call(self, x):
    """
    Description:
    ---------------
    Similar to the forward function in tensorflow, this function is used to train the AI.

    Input(s)
    ---------------
    x : tf.Tensor

    Output(s)
    ---------------
    output: tf.Tensor
    """
    print("----------------------entrée dans le call de DropPath------------------------")
    print("x = {}".format(x))
    print("self._trainable = {}".format(self._trainable))
    print("self.p = {}".format(self.p))
    print("self.p>0 = {}".format(self.p>0))
    print("self._trainable and self.p>0 = {}".format(self._trainable and self.p>0))
    if self._trainable and self.p>0:
      keep_prob = 1. - self.p
      print("keep_prob = {}".format(keep_prob))
      print("tf.shape(x) = {}".format(tf.shape(x)))
      print("len(tf.shape(x)) = {}".format(len(tf.shape(x))))
      print("len(tf.shape(x)) - 1 == {}".format(len(tf.shape(x)) - 1))
      print("(1,) * (len(tf.shape(x)) - 1) = {}".format((1,) * (len(tf.shape(x)) - 1)))
      print("tf.shape(x)[0] = {}".format(tf.shape(x)[0]))
      print("(tf.shape(x)[0],) = {}".format((tf.shape(x)[0],)))
      shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
      print("shape = {}".format(shape))
      print("tf.float32 = {}".format(tf.float32))
      mask = tf.keras.backend.random_bernoulli(shape, keep_prob, dtype = tf.float32)
      print("mask = {}".format(mask))
      print("x/keep_prob*mask = {}".format(x/keep_prob*mask))
      return x/keep_prob*mask
    return x


### Code tensorflow de la classe PoolBN

In [135]:
import tensorflow as tf


class PoolBN(tf.keras.layers.Layer):
  """
  Description
  ---------------
  This class is used to handle both the max pooling layer and the average pooling layer.
  In both cases, the pooling is followed by a layer of batch normalization.
  """
  def __init__(self, pool_type, C, kernel_size, stride, padding, affine=False):
    """
    Description
    ---------------
    Creates an instance of the class PoolBN

    Input(s)
    ---------------
    pool_type: string (either "max" or "avg")
    C: int, axis on which to perform batch normalization
    kernel_size: int or enumerable of ints
    stride: int or enumerable of ints
    padding: string
    affine: bool

    Output(s)
    ---------------
    No output
    """
    print("-------------------entrée dans la fonction __init__ de la classe PoolBN ------------------------")
    super().__init__()
    print("pool_type = {}".format(pool_type))
    print("C = {}".format(C))
    print("kernel_size = {}".format(kernel_size))
    print("stride = {}".format(stride))
    print("padding = {}".format(padding))
    print("affine = {}".format(affine))
    if pool_type == "max":
      self.pooling = tf.keras.layers.MaxPooling2D(kernel_size, stride, padding)
    elif pool_type == "avg":
      self.pooling = tf.keras.layers.AveragePooling2D(kernel_size, stride, padding)
    else:
      raise ValueError()
    self.batchnormalisation = tf.keras.layers.BatchNormalization()
  
  def call(self, x):
    """
    Description:
    ---------------
    Similar to the forward function in tensorflow, this function is used to train the AI.

    Input(s)
    ---------------
    x : tf.Tensor

    Output(s)
    ---------------
    output: tf.Tensor
    """
    print("-----------entrée dans la fonction call de PoolBN--------------")
    print("x = {}".format(x))
    print("self.pooling = {}".format(self.pooling))
    if isinstance(x,tf.Tensor):
      # x is a Tensor
      print("x.get_shape() = {}".format(x.get_shape()))
      out1 = self.pooling(x)
      print("out1 = {}".format(out1))
      print("out1.get_shape() = {}".format(out1.get_shape()))
      out = self.batchnormalisation(out1)
      print("out.get_shape() = {}".format(out.get_shape()))
    elif x == ():
      print("x is empty")
      out = ()
    else:
      # x is a list
      out = ()
      print("x[0] = {}".format(x[0]))
      print("len(x) = {}".format(len(x)))
      for i in range(len(x)):
        print("i_pool_bn = {}".format(i))
        print("len(x[i_pool_bn]) = {}".format(len(x[i])))
        if isinstance(len(x[i]),int):
          # we want to know if x[i] is a tensor list
          if len(x[i]) == 0 :
            # x[i] contains no element
            print("x[i_pool_bn] est vide")
          elif len(x[i]) == 1 :
            # x[i] contains only one tensor
            entree = x[i][0]
            print("entree.get_shape() = {}".format(entree.get_shape()))
            x_i = self.pooling(entree)
            print("x_i_pool_bn = {}".format(x_i))
            print("x_i_pool_bn.get_shape() = {}".format(x_i.get_shape()))
            print("x_i_pool_bn.get_shape()[1] = {}".format(x_i.get_shape()[1]))
            print("self.batchnormalisation = {}".format(self.batchnormalisation))
            out_i = self.batchnormalisation(x_i)
            print("out_i_pool_bn.get_shape() = {}".format(out_i.get_shape()))
            out = out + (out_i,)
          else:
            # x[i] is a list containing more than one tensor
            print("x[i] = {}".format(x[i]))
            for j in range(len(x[i])):
              entree = x[i][j]
              print("entree = {}".format(entree))
              print("entree.get_shape() = {}".format(entree.get_shape()))
              print("entree.get_shape()[0] = {}".format(entree.get_shape()[0]))
              print("entree.get_shape()[1] = {}".format(entree.get_shape()[1]))
              print("entree.get_shape()[2] = {}".format(entree.get_shape()[2]))
              print("entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3 = {}".format(entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3))
              if entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3:
                # We check that the dimensions of the tensor respect the conditions necessary to apply the layer.
                x_i_j = self.pooling(entree)
                print("x_i_pool_bn_j = {}".format(x_i_j))
                print("self.batchnormalisation = {}".format(self.batchnormalisation))
                print("x_i_pool_bn_j.get_shape() = {}".format(x_i_j.get_shape()))
                print("x_i_pool_bn_j.get_shape()[0] = {}".format(x_i_j.get_shape()[0]))
                print("x_i_pool_bn_j.get_shape()[1] = {}".format(x_i_j.get_shape()[1]))
                print("x_i_pool_bn_j.get_shape()[2] = {}".format(x_i_j.get_shape()[2]))
                print("x_i_pool_bn_j.get_shape()[3] = {}".format(x_i_j.get_shape()[3]))
                out_i_j = self.batchnormalisation(x_i_j)
                print("out_i_pool_bn_j = {}".format(out_i_j))
                print("out_i_pool_bn_j.get_shape() = {}".format(out_i_j.get_shape()))
                out = out + (out_i_j,)
              else:
                out_i_j = tf.constant(1,shape=(3, 3, 3, 4))
                print("out_i_pool_bn_j = {}".format(out_i_j))
                print("out_i_pool_bn_j.get_shape() = {}".format(out_i_j.get_shape()))
                out = out + (out_i_j,)            
        else:
          # x[i] is a Tensor
          x_i = self.pooling(x[i])
          print("x_i_pool_bn = {}".format(x_i))
          print("self.batchnormalisation = {}".format(self.batchnormalisation))
          out_i = self.batchnormalisation(x_i)
          print("out_i_pool_bn = {}".format(out_i))
          out = out + (out_i,)
    print("out = {}".format(out))
    return out


### Code tensorflow de la classe StdConv

In [127]:
import tensorflow as tf

class StdConv(tf.keras.layers.Layer):
  """
  Description
  ---------------
  This class is used to do an average convolution.
  It is composed of a relu followed by the convolution.
  Then a batchnormalisation is applied.
  The model used is a sequential keras model.
  """
  def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
    """
    Description
    ---------------
    Creates an instance of the class StdConv

    Input(s)
    ---------------
    C_in: int
    C_out: int
    kernel_size: int
    stride: int
    padding: string ("same" ou "valid")
    affine: bool

    Output(s)
    ---------------
    No output
    """
    print("-----------entrée dans la fonction __init__ de la classe StdConv")
    super().__init__()
    self.relu = tf.keras.layers.ReLU()
    self.conv2d = tf.keras.layers.Conv2D(filters=C_out,
                                        kernel_size=kernel_size,
                                        strides=stride,
                                        padding=padding, 
                                        use_bias=False )
    self.batch_normalisation = tf.keras.layers.BatchNormalization(axis=-1)
  
  def call(self, x):
    """
    Description:
    ---------------
    Similar to the forward function in tensorflow, this function is used to train the AI.

    Input(s)
    ---------------
    x : tf.Tensor

    Output(s)
    ---------------
    output: tf.Tensor
    """
    print("-----------------entrée dans la fonction call de la classe StdConv --------------")
    print("x = {}".format(x))
    if isinstance(x,tf.Tensor):
      # x is a Tensor
      print("x.get_shape() = {}".format(x.get_shape()))
      out_relu = self.relu(x)
      print("out_relu = {}".format(out_relu))
      print("out_relu.get_shape()".format(out_relu.get_shape()))
      out_conv2d = self.conv2d(out_relu)
      print("out_conv2d = {}".format(out_conv2d))
      print("out_conv2d.get_shape() = {}".format(out_conv2d.get_shape()))
      out_batch_normalisation = self.batch_normalisation(out_conv2d)
      print("out_batch_normalisation = {}".format(out_batch_normalisation))
      print("out_batch_normalisation.get_shape() = {}".format(out_batch_normalisation.get_shape()))
      out = out_batch_normalisation
    else:
      # x is a list
      out = ()
      print("x[0] = {}".format(x[0]))
      print("len(x) = {}".format(len(x)))
      for i in range(len(x)):
        print("i_std_conv = {}".format(i))
        print("len(x[i_std_conv]) = {}".format(len(x[i])))
        if isinstance(len(x[i]),int):
          # we want to know if x[i] is a tensor list
          if len(x[i]) == 0 :
            # x[i] contains no element
            print("x[i_std_conv] est vide")
          elif len(x[i]) == 1 :
            # x[i] contains only one tensor
            entree = x[i][0]
            print("entree.get_shape() = {}".format(entree.get_shape()))
            x_i = self.relu(entree)
            print("x_i_std_conv = {}".format(x_i))
            print("x_i_std_conv.get_shape() = {}".format(x_i.get_shape()))
            print("x_i_std_conv.get_shape()[1] = {}".format(x_i.get_shape()[1]))
            out_conv2d_i = self.conv2d(x_i)
            print("out_conv2d_i_std_conv.get_shape() = {}".format(out_conv2d_i.get_shape()))
            out_i = self.batch_normalisation(x_i)
            print("out_i_std_conv.get_shape() = {}".format(out_i.get_shape()))
            out = out + (out_i,)
          else:
            # x[i] is a list containing more than one tensor
            for j in range(len(x[i])):
              entree = x[i][j]
              print("entree = {}".format(entree))
              print("entree.get_shape() = {}".format(entree.get_shape()))
              x_i_j = self.relu(entree)
              print("x_i_std_conv_j = {}".format(x_i_j))
              print("x_i_std_conv_j.get_shape() = {}".format(x_i_j.get_shape()))
              print("x_i_std_conv_j.get_shape()[1] = {}".format(x_i_j.get_shape()[1]))
              out_conv2d_i_j = self.conv2d(x_i_j)
              print("out_conv2d_i_std_conv_j.get_shape() = {}".format(out_conv2d_i_j.get_shape()))
              out_i_j = self.batch_normalisation(out_conv2d_i_j)
              print("out_i_std_conv_j = {}".format(out_i_j))
              print("out_i_std_conv_j.get_shape() = {}".format(out_i_j.get_shape()))
              out = out + (out_i_j,)
            
        else:
          # x[i] is a Tensor
          x_i = self.relu(x[i])
          print("x_i_std_conv = {}".format(x_i))
          print("x_i_std_conv.get_shape()".format(x_i.get_shape()))
          out_conv2d_i = self.conv2d(x_i)
          print("out_conv2d_i_std_conv = {}".format(out_conv2d_i))
          print("out_conv2d_i_std_conv.get_shape() = {}".format(out_conv2d_i.get_shape()))
          out_i = self.batch_normalisation(out_conv2d_i)
          print("out_i_std_conv = {}".format(out_i))
          print("out_i_std_conv.get_shape() = {}".format(out_i.get_shape()))
          out = out + (out_i,)
    print("out = {}".format(out))
    return out



### test des fonctions créées

#### Test de la classe DilConv

In [108]:
test_tensor = tf.ones((10,10,10,10))
model = Sequential()
#model.add(tf.keras.layers.ReLU())
#model.add(FactorizedReduce(2,1))
model.add(DilConv(1,1,1,1,1,1))
#model.add(SepConv(1,1,1,1,1))

model.compile(loss="mse")
model.fit(test_tensor, tf.zeros((10,10,10,10)))

1/1 [==============================] - 1s 787ms/step - loss: 5.1537e-11


#### Test de la classe SepConv

In [109]:
test_tensor = tf.ones((10,10,10,10))
model = Sequential()
#model.add(tf.keras.layers.ReLU())
#model.add(FactorizedReduce(2,1))
#model.add(DilConv(1,1,1,1,1,1))
model.add(SepConv(1,1,1,1,1))

model.compile(loss="mse")
model.fit(test_tensor, tf.zeros((10,10,10,10)))

1/1 [==============================] - 1s 1s/step - loss: 5.7425e-12


#### Test de la classe FactorizedReduce

In [110]:
"""
import tensorflow as tf
import tensorflow.keras as K
nb_input = 10
test_tensors = tf.ones((nb_input,12,12,4))
test_model = K.models.Sequential()
# test_pool = PoolBN("max", 3, 3, 1, "same")
test_model.add(PoolBN("max", 3, 3, 1, "same"))
# test_model.add(PoolBN("max", 3, 3, 2, "same"))
test_model.add(FactorizedReduce(3,3))
test_model.add(drop_path(0.3))
# test_model.add(SepConv(3,3,3,1,'valid'))

# test_model.add(Layer()) # this is apparently a identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,5,5,2)))
test_model.summary()
"""

'\nimport tensorflow as tf\nimport tensorflow.keras as K\nnb_input = 10\ntest_tensors = tf.ones((nb_input,12,12,4))\ntest_model = K.models.Sequential()\n# test_pool = PoolBN("max", 3, 3, 1, "same")\ntest_model.add(PoolBN("max", 3, 3, 1, "same"))\n# test_model.add(PoolBN("max", 3, 3, 2, "same"))\ntest_model.add(FactorizedReduce(3,3))\ntest_model.add(drop_path(0.3))\n# test_model.add(SepConv(3,3,3,1,\'valid\'))\n\n# test_model.add(Layer()) # this is apparently a identity layer\ntest_model.compile(loss = "mse")\ntest_model.fit(test_tensors, tf.ones((nb_input,5,5,2)))\ntest_model.summary()\n'

#### Test de la classe StdConv

In [111]:
import tensorflow as tf
nb_input = 10
test_tensors = tf.ones((nb_input,12,12,4))
test_model = tf.keras.models.Sequential()
test_model.add(StdConv(3 , 3, 3,  1,"same"))
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,12,12,3)))
test_model.summary()

-----------entrée dans la fonction __init__ de la classe StdConv
-----------------entrée dans la fonction call de la classe StdConv --------------
x = Tensor("Placeholder:0", shape=(None, 12, 12, 4), dtype=float32)
x.get_shape() = (None, 12, 12, 4)
out_relu = Tensor("std_conv_2/re_lu_20/Relu:0", shape=(None, 12, 12, 4), dtype=float32)
out_relu.get_shape()
out_conv2d = Tensor("std_conv_2/conv2d_32/Conv2D:0", shape=(None, 12, 12, 3), dtype=float32)
out_conv2d.get_shape() = (None, 12, 12, 3)
out_batch_normalisation = Tensor("std_conv_2/batch_normalization_19/FusedBatchNormV3:0", shape=(None, 12, 12, 3), dtype=float32)
out_batch_normalisation.get_shape() = (None, 12, 12, 3)
out = Tensor("std_conv_2/batch_normalization_19/FusedBatchNormV3:0", shape=(None, 12, 12, 3), dtype=float32)
-----------------entrée dans la fonction call de la classe StdConv --------------
x = Tensor("IteratorGetNext:0", shape=(None, 12, 12, 4), dtype=float32)
x.get_shape() = (None, 12, 12, 4)
out_relu = Tensor("seque

# Conversion classe fichier model.py en fichier tensorflow

## Installation du module tensorflow_lattice

In [112]:
!pip install tensorflow_lattice

## Dummies des fonctions LayerChoice et InputChoice utilisées pour le test

### Dummy de la fonction LayerChoice

In [113]:
# dummy of the function LayerChoice

import tensorflow as tf
@Trace_appel()
def LayerChoice(dictionnaire, label:int):
  print("----------------entrée dans LayerChoice-----------------------")
  print("dictionnaire = {}".format(dictionnaire))
  print("label = {}".format(label))
  channels = 4
  print("channels = {}".format(channels))
  stride = 1
  print("stride  = {}".format(stride))
  choix = 1
  print("choix = {}".format(choix))
  if choix == 1:
    print("layer chosen : Max pooling")
    layer_chosen = PoolBN('max', channels, 3, stride, "same", affine=False)
  elif choix == 2:
    print("layer chosen : average pooling")
    layer_chosen = PoolBN('avg', channels, 3, stride, "same", affine=False)
  elif choix == 3:
    print("layer chosen : skip connect")
    print("stride = {}".format(stride))
    if stride == 1:
      layer_chosen = tf.identity(channels) 
    else:
       layer_chosen = FactorizedReduce(channels, channels, affine=False)
  elif choix == 4:
    print("layer chosen : sepconv3x3")
    layer_chosen = SepConv(channels, channels, 3, stride, 1, affine=False)
  elif choix == 5:
    print("layer chosen : sepconv5x5")
    layer_chosen = SepConv(channels, channels, 5, stride, 2, affine=False)
  elif choix == 6:
    print("layer chosen : dilconv3x3")
    layer_chosen = DilConv(channels, channels, 3, stride, 2, 2, affine=False)
  elif choix == 7:
    print("layer chosen : dilconv5x5")
    layer_chosen = DilConv(channels, channels, 5, stride, 4, 2, affine=False)
  else:
    print("no layer chosen")
  print("layer_chosen = {}".format(layer_chosen))
  return layer_chosen

### Dummy de la fonction InputChoice

In [114]:
# dummy of the function InputChoice

import tensorflow as tf
@Trace_appel()
def InputChoice( n_candidates, n_chosen, label:str,
                 reduction=None):
  print("------------------------entrée dans InputChoice---------------------------")
  channels = 4
  stride = 1
  input_chosen = PoolBN('max', channels, 3, stride, "valid", affine=False)
  print("label[1] = {}".format(label[1]))
  #input_chosen = tf.keras.layers.concatenate(n_candidates)
  return input_chosen


### Vrai fonction choices_utils.py

In [115]:
from typing import Any, Dict, List, Optional
from collections import defaultdict


def get_fixed_value(label: str) -> Any:
    ret = get_current_context('fixed')
    try:
        return ret[generate_new_label(label)]
    except KeyError:
        raise KeyError(f'Fixed context with {label} not found. Existing values are: {ret}')


def get_current_context(key: str) -> Any:
    return ContextStack.top(key)

### vrai classe ContextStack du fichier choices_utils.py

In [116]:
class ContextStack:
    _stack: Dict[str, List[Any]] = defaultdict(list)

    def __init__(self, key: str, value: Any):
        self.key = key
        self.value = value

    def __enter__(self):
        self.push(self.key, self.value)
        return self

    def __exit__(self, *args, **kwargs):
        self.pop(self.key)

    @classmethod
    def push(cls, key: str, value: Any):
        cls._stack[key].append(value)

    @classmethod
    def pop(cls, key: str) -> None:
        cls._stack[key].pop()

    @classmethod
    def top(cls, key: str) -> Any:
        return cls._stack[key][-1]

### dernière fonction fichier choices_utils.py

In [117]:
def generate_new_label(label: Optional[str]):
    if label is None:
        return "model"
    return label

### Vrai code classe LayerChoice du fichier choices.py

In [118]:
import tensorflow as tf
from typing import Union, Dict, List, Optional, OrderedDict
import warnings

class LayerChoice(tf.Module):

    def create_fixed_module(cls, candidates: Union[Dict[str, tf.Module], List[tf.Module]], *,
                            label: Optional[str] = None, **kwargs):
        chosen = get_fixed_value(label) # choices_utils.get_fixed_value(label)
        if isinstance(candidates, list):
            return candidates[int(chosen)]
        else:
            return candidates[chosen]

    def __init__(self, candidates: Union[Dict[str, tf.Module], List[tf.Module]], *,
                 prior: Optional[List[float]] = None, label: Optional[str] = None, **kwargs):
                 
        super(LayerChoice, self).__init__()
        if 'key' in kwargs:
            warnings.warn(f'"key" is deprecated. Assuming label.')
            label = kwargs['key']
        if 'return_mask' in kwargs:
            warnings.warn(f'"return_mask" is deprecated. Ignoring...')
        if 'reduction' in kwargs:
            warnings.warn(f'"reduction" is deprecated. Ignoring...')
        self.candidates = candidates
        self.prior = prior or [1 / len(candidates) for _ in range(len(candidates))]
        assert abs(sum(self.prior) - 1) < 1e-5, 'Sum of prior distribution is not 1.'
        self._label = generate_new_label(label) # choices_utils.generate_new_label(label)
        self._modules: Dict[str, Optional[tf.Module]] = OrderedDict()

        self.names = []
        if isinstance(candidates, dict):
            for name, module in candidates.items():
                assert name not in ["length", "reduction", "return_mask", "_key", "key", "names"], \
                    "Please don't use a reserved name '{}' for your module.".format(name)
                self._modules[name] = module
                self.names.append(name)
        elif isinstance(candidates, list):
            for i, module in enumerate(candidates):
                self._modules[name] = module
                self.names.append(str(i))
        else:
            raise TypeError("Unsupported candidates type: {}".format(type(candidates)))
        self._first_module = self._modules[self.names[0]]  # to make the dummy forward meaningful

    @property
    def key(self):
        return self._key()

    def _key(self):
        warnings.warn('Using key to access the identifier of LayerChoice is deprecated. Please use label instead.',
                      category=DeprecationWarning)
        return self._label

    @property
    def label(self):
        return self._label

    def __getitem__(self, idx):
        if isinstance(idx, str):
            return self._modules[idx]
        return list(self)[idx]

    def __setitem__(self, idx, module):
        key = idx if isinstance(idx, str) else self.names[idx]
        return setattr(self, key, module)

    def __delitem__(self, idx):
        if isinstance(idx, slice):
            for key in self.names[idx]:
                delattr(self, key)
        else:
            if isinstance(idx, str):
                key, idx = idx, self.names.index(idx)
            else:
                key = self.names[idx]
            delattr(self, key)
        del self.names[idx]

    def __len__(self):
        return len(self.names)

    def __iter__(self):
        return map(lambda name: self._modules[name], self.names)

    @property
    def choices(self):
        return self._choices()

    def _choices(self):
        warnings.warn("layer_choice.choices is deprecated. Use `list(layer_choice)` instead.", category=DeprecationWarning)
        return list(self)

    def __call__(self, x):
        warnings.warn('You should not run forward of this module directly.')
        return self._first_module(x)

    def __repr__(self):
        return f'LayerChoice({self.candidates}, label={repr(self.label)})'

#### Texte code LayerChoice



```
# Ce texte est au format code
```

import tensorflow as tf
from typing import Union, Dict, List, Optional, OrderedDict
import warnings

class LayerChoice(tf.Module):

    def create_fixed_module(cls, candidates: Union[Dict[str, tf.Module], List[tf.Module]], *,
                            label: Optional[str] = None, **kwargs):
        chosen = get_fixed_value(label) # choices_utils.get_fixed_value(label)
        if isinstance(candidates, list):
            return candidates[int(chosen)]
        else:
            return candidates[chosen]

    def __init__(self, candidates: Union[Dict[str, tf.Module], List[tf.Module]], *,
                 prior: Optional[List[float]] = None, label: Optional[str] = None, **kwargs):
                 
        super(LayerChoice, self).__init__()
        if 'key' in kwargs:
            warnings.warn(f'"key" is deprecated. Assuming label.')
            label = kwargs['key']
        if 'return_mask' in kwargs:
            warnings.warn(f'"return_mask" is deprecated. Ignoring...')
        if 'reduction' in kwargs:
            warnings.warn(f'"reduction" is deprecated. Ignoring...')
        self.candidates = candidates
        self.prior = prior or [1 / len(candidates) for _ in range(len(candidates))]
        assert abs(sum(self.prior) - 1) < 1e-5, 'Sum of prior distribution is not 1.'
        self._label = generate_new_label(label) # choices_utils.generate_new_label(label)
        self._modules: Dict[str, Optional[tf.Module]] = OrderedDict()

        self.names = []
        if isinstance(candidates, dict):
            for name, module in candidates.items():
                assert name not in ["length", "reduction", "return_mask", "_key", "key", "names"], \
                    "Please don't use a reserved name '{}' for your module.".format(name)
                self._modules[name] = module
                self.names.append(name)
        elif isinstance(candidates, list):
            for i, module in enumerate(candidates):
                self._modules[name] = module
                self.names.append(str(i))
        else:
            raise TypeError("Unsupported candidates type: {}".format(type(candidates)))
        self._first_module = self._modules[self.names[0]]  # to make the dummy forward meaningful

    @property
    def key(self):
        return self._key()

    def _key(self):
        warnings.warn('Using key to access the identifier of LayerChoice is deprecated. Please use label instead.',
                      category=DeprecationWarning)
        return self._label

    @property
    def label(self):
        return self._label

    def __getitem__(self, idx):
        if isinstance(idx, str):
            return self._modules[idx]
        return list(self)[idx]

    def __setitem__(self, idx, module):
        key = idx if isinstance(idx, str) else self.names[idx]
        return setattr(self, key, module)

    def __delitem__(self, idx):
        if isinstance(idx, slice):
            for key in self.names[idx]:
                delattr(self, key)
        else:
            if isinstance(idx, str):
                key, idx = idx, self.names.index(idx)
            else:
                key = self.names[idx]
            delattr(self, key)
        del self.names[idx]

    def __len__(self):
        return len(self.names)

    def __iter__(self):
        return map(lambda name: self._modules[name], self.names)

    @property
    def choices(self):
        return self._choices()

    def _choices(self):
        warnings.warn("layer_choice.choices is deprecated. Use `list(layer_choice)` instead.", category=DeprecationWarning)
        return list(self)

    def __call__(self, x):
        warnings.warn('You should not run forward of this module directly.')
        return self._first_module(x)

    def __repr__(self):
        return f'LayerChoice({self.candidates}, label={repr(self.label)})'

### vrai code classe InputChoice du fichier choices.py

In [119]:
import tensorflow as tf
from typing import Union, Dict, List, Optional, OrderedDict
import warnings
# choices_utils.get_fixed_value(label)

class InputChoice(tf.Module):
  
    @classmethod
    def create_fixed_module(cls, n_candidates: int, n_chosen: Optional[int] = 1, reduction: str = 'sum', *,
                            prior: Optional[List[float]] = None, label: Optional[str] = None, **kwargs):
        return ChosenInputs(get_fixed_value(label), reduction=reduction)

    def __init__(self, n_candidates: int, n_chosen: Optional[int] = 1,
                 reduction: str = 'sum', *,
                 prior: Optional[List[float]] = None, label: Optional[str] = None, **kwargs):
        super(InputChoice, self).__init__()
        if 'key' in kwargs:
            warnings.warn(f'"key" is deprecated. Assuming label.')
            label = kwargs['key']
        if 'return_mask' in kwargs:
            warnings.warn(f'"return_mask" is deprecated. Ignoring...')
        if 'choose_from' in kwargs:
            warnings.warn(f'"reduction" is deprecated. Ignoring...')
        self.n_candidates = n_candidates
        self.n_chosen = n_chosen
        self.reduction = reduction
        self.prior = prior or [1 / n_candidates for _ in range(n_candidates)]
        assert self.reduction in ['mean', 'concat', 'sum', 'none']
        self._label = generate_new_label(label) # choices_utils.generate_new_label(label)

    @property
    def key(self):
        return self._key()

    def _key(self):
        warnings.warn('Using key to access the identifier of InputChoice is deprecated. Please use label instead.',
                      category=DeprecationWarning)
        return self._label

    @property
    def label(self):
        return self._label

    def __call__(self, candidate_inputs: List[tf.Tensor]) -> tf.Tensor:
        warnings.warn('You should not run forward of this module directly.')
        return candidate_inputs[0]

    def __repr__(self):
        return f'InputChoice(n_candidates={self.n_candidates}, n_chosen={self.n_chosen}, ' \
            f'reduction={repr(self.reduction)}, label={repr(self.label)})'

### Vrai code classe ChosenInputs du fichier choices.py

In [120]:
import tensorflow as tf
from typing import Union, Dict, List, Optional, OrderedDict
import warnings

class ChosenInputs(tf.Module):
    """
    A module that chooses from a tensor list and outputs a reduced tensor.
    The already-chosen version of InputChoice.
    """

    def __init__(self, chosen: Union[List[int], int], reduction: str):
        super().__init__()
        self.chosen = chosen if isinstance(chosen, list) else [chosen]
        self.reduction = reduction

    def __call__(self, candidate_inputs):
        return self._tensor_reduction(self.reduction, [candidate_inputs[i] for i in self.chosen])

    def _tensor_reduction(self, reduction_type, tensor_list):
        if reduction_type == 'none':
            return tensor_list
        if not tensor_list:
            return None  # empty. return None for now
        if len(tensor_list) == 1:
            return tensor_list[0]
        if reduction_type == 'sum':
            return sum(tensor_list)
        if reduction_type == 'mean':
            return sum(tensor_list) / len(tensor_list)
        if reduction_type == 'concat':
            return tf.cat(tensor_list, dim=1)
        raise ValueError(f'Unrecognized reduction policy: "{reduction_type}"')

## conversion de la classe AuxiliaryHead en classe compatible avec tensorflow

In [121]:
import tensorflow as tf


class AuxiliaryHead(tf.keras.layers.Layer):
  """
  Description
  ---------------
  This class allows the creation of an auxiliary head that takes 2/3 of the
  place of network to let the gradient flow well .
  """
  def __init__(self, input_size, C, n_classes):
    """
    Description
    ---------------
    This function creates an instance of the AuxiliaryHead class.
    We are assuming that the input size is either 7x7 or 8x8.
    Input(s)
    ---------------
    input_size: int
    C: int
    n_classes: int
    Output(s)
    ---------------
    no output
    """
    print("-------------entrée dans l'init de la classe AuxiliaryHead ---------------------")
    print("input_size = {}".format(input_size))
    assert input_size in [7, 8]
    print("input_size in [7, 8] = {}".format(input_size in [7, 8]))
    print("----------après l'assert de l'init de AuxiliaryHead----------")
    super().__init__()
    self.net = tf.keras.Sequential()
    self.net.add(tf.keras.layers.ReLU())
    self.net.add(tf.keras.AvgPooling2D(5, 
                                       stride=input_size - 5,
                                       padding=0,
                                       count_include_pad=False)) # 2x2 out
    self.net.add(tf.keras.layers.Conv2D(filters=128, 
                                        kernel_size=C, 
                                        use_bias=False, 
                                        data_format="channels_last"))
    self.net.add(tf.keras.layers.BatchNormalization(axis=-1))
    self.net.add(tf.keras.layers.ReLU())
    self.net.add(tf.keras.layers.Conv2D(filters=768, 
                                        kernel_size= 128,
                                        data_format="channels_last",
                                        use_bias=False)) # 1x1 out
    self.net.add(tf.keras.layers.BatchNormalization(axis=-1))
    self.net.add(tf.keras.layers.ReLU())
    print("self.net = {}".format(self.net))
    self.linear = tf.layers.Linear(768, n_classes)
    print("self.linear = {}".format(self.linear))


  def call(self, x):
    """
    Description
    ---------------
    This function applies the instance of the AuxiliaryHead class to the data.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    logits : tf.Tensor
    """
    print("------------entrée dans le call de la fonction AuxiliaryHead--------------")
    print("x = {}".format(x))
    print("self.net = {}".format(self.net))
    out = self.net(x)
    print("out = {}".format(out))
    print("out.view = {}".format(out.view))
    print("out.size(0) = {}".format(out.size(0)))
    out = out.view(out.size(0), -1)  # flatten
    print("out = {}".format(out))
    print("self.linear = {}".format(self.linear))
    logits = self.linear(out)
    print("logits = {}".format(logits))
    return logits

## classe Node en tensorflow

### code à rajouter dans le fichier model.py

In [122]:
"""
import tensorflow as tf
from collections import OrderedDict
import ops

class Node(tf.keras.layers.Layer):
  def __init__(self, node_id, num_prev_nodes, channels, num_downsample_connect):
"""
    
"""
    super().__init__()
    self.ops = []
    choice_keys = []
    for i in range(num_prev_nodes):
      stride = 2 if i < num_downsample_connect else 1
      choice_keys.append("{}_p{}".format(node_id, i))
      self.ops.append(
          LayerChoice(OrderedDict([
                                   ("maxpool", ops.PoolBN('max', channels, 3, stride, 1, affine=False)),
                                   ("avgpool", ops.PoolBN('avg', channels, 3, stride, 1, affine=False)),
                                   ("skipconnect", nn.Identity() if stride == 1 else ops.FactorizedReduce(channels, channels, affine=False)),
                                   ("sepconv3x3", ops.SepConv(channels, channels, 3, stride, 1, affine=False)),
                                   ("sepconv5x5", ops.SepConv(channels, channels, 5, stride, 2, affine=False)),
                                   ("dilconv3x3", ops.DilConv(channels, channels, 3, stride, 2, 2, affine=False)),
                                   ("dilconv5x5", ops.DilConv(channels, channels, 5, stride, 4, 2, affine=False))
                                   ]), label=choice_keys[-1]))
      self.drop_path = ops.DropPath()
      self.input_switch = InputChoice(n_candidates=len(choice_keys),
                                      n_chosen=2,
                                      label="{}_switch".format(node_id))

  def call(self, prev_nodes):
"""
    
"""
    assert len(self.ops) == len(prev_nodes)
    out = [op(node) for op, node in zip(self.ops, prev_nodes)]
    out = [self.drop_path(o) if o is not None else None for o in out]
    return self.input_switch(out)
"""

'\n    assert len(self.ops) == len(prev_nodes)\n    out = [op(node) for op, node in zip(self.ops, prev_nodes)]\n    out = [self.drop_path(o) if o is not None else None for o in out]\n    return self.input_switch(out)\n'

### Code de test Node

In [123]:
from tensorflow.python import ops
from tensorflow.python import ops
import tensorflow as tf
from collections import OrderedDict
import tensorflow_lattice as tfl

class Node(tf.keras.layers.Layer):
  """
  Description
  ---------------
  This class allows the creation of an node.
  """
  @Trace_appel()
  def __init__(self, node_id, num_prev_nodes, channels, num_downsample_connect):
    """
    Description
    ---------------
    This function creates an instance of the Node class.
    Input(s)
    ---------------
    node_id: int
    num_prev_nodes: int
    channels: int
    num_downsample_connect: int
    Output(s)
    ---------------
    no output
    """
    print("----------------------entrée dans la fonction init de Node---------------------")
    super().__init__()
    self.ops = []
    choice_keys = []
    print("num_prev_nodes = {}".format(num_prev_nodes))
    for i in range(num_prev_nodes):
      print("i = {}".format(i))
      print("i < num_downsample_connect = {}".format(i < num_downsample_connect))
      stride = 2 if i < num_downsample_connect else 1
      print("stride = {}".format(stride))
      choice_keys.append("{}_p{}".format(node_id, i))
      print("choice_keys = {}".format(choice_keys))
      self.ops.append(
          LayerChoice(OrderedDict([
                                   ("maxpool", PoolBN('max', channels, 3, stride, "same", affine=False)),
                                   ("avgpool", PoolBN('avg', channels, 3, stride, "same", affine=False)),
                                   ("skipconnect",
                                    tf.identity(channels) if stride == 1 else FactorizedReduce(channels,
                                                                                               channels,
                                                                                               affine=False)),
                                   ("sepconv3x3", SepConv(channels,
                                                          channels,
                                                          3,
                                                          stride,
                                                          1,
                                                          affine=False)),
                                   ("sepconv5x5",
                                    SepConv(channels,
                                            channels,
                                            5,
                                            stride,
                                            2,
                                            affine=False)),
                                   ("dilconv3x3",
                                    DilConv(channels,
                                            channels,
                                            3, stride, 2, 2, affine=False)),
                                   ("dilconv5x5",
                                    DilConv(channels,
                                            channels,
                                            5, stride, 4, 2, affine=False))
                                   ]), label=choice_keys[-1]))
      print("self.ops = {}".format(self.ops))
      self.drop_path = DropPath()
      print("choice_keys = {}".format(choice_keys))
      print("n_candidates = {}".format(len(choice_keys)))
      print("n_chosen = {}".format(2))
      print("label = {}".format(node_id))
      self.input_switch = InputChoice(n_candidates=len(choice_keys),
                                      n_chosen=2,
                                      label="{}_switch".format(node_id))
      print("self.input_switch = {}".format(self.input_switch))

  #@Trace_appel()
  def call(self, prev_nodes):
    """
    Description
    ---------------
    This function applies the instance of the Node class to the previous instances of the class Node.
    Input(s)
    ---------------
    prev_nodes: Node ( it's the previously created instances of the Node class)
    Output(s)
    ---------------
    self.input_switch(out) : 
    """
    print("--------entrée dans le call de la classe Node -------")
    print("prev_nodes = {}".format(prev_nodes))
    print("len(prev_nodes) = {}".format(len(prev_nodes)))
    print("self.ops = {}".format(self.ops))
    print("len(self.ops) = {}".format(len(self.ops)))
    assert len(self.ops) == len(prev_nodes)
    print("len(self.ops) == len(prev_nodes) = {}".format(len(self.ops) == len(prev_nodes)))
    print("--------après assert--------")
    print("zip(self.ops, prev_nodes) = {}".format(zip(self.ops, prev_nodes)))
    out = [op(node) for op, node in zip(self.ops, prev_nodes)]
    print("first_out_before_drop_path = {}".format(out))
    out = [self.drop_path(o) if o is not None else None for o in out]
    print("out = {}".format(out))
    print("self.input_switch = {}".format(self.input_switch))
    print("self.input_switch(out) = {}".format(self.input_switch(out)))
    return self.input_switch(out)


### test code node

In [129]:
import tensorflow as tf
print("=====================Test de la classe Node ==============================")
nb_input = 10
print("nb_input = {}".format(nb_input))
test_tensors = tf.ones((nb_input,32,32,4))
print("isinstance(test_tensors, tf.Tensor) = {}".format(isinstance(test_tensors, tf.Tensor)))
#print("test_tensors = {}".format(test_tensors))
test_model = tf.keras.models.Sequential()
test_model.add(Node("Reduce_n1",1,4,0))
print("test_model = {}".format(test_model))
test_model.compile(loss = "mse")
test_model.fit([test_tensors], tf.ones((nb_input,32,32,4)))
test_model.summary()

INFO:
INFO:>>>>>> Appel de __init__ >>>>>>
DEBUG:  ... Arguments :  N°1=<__main__.Node object at 0x7f956d761350>   N°2=Reduce_n1   N°3=1   N°4=4   N°5=0 
DEBUG:  ... Resultat : None
INFO:----- Fin de __init__ -----


=====================Test de la classe Node ==============================
nb_input = 10
isinstance(test_tensors, tf.Tensor) = True
----------------------entrée dans la fonction init de Node---------------------
num_prev_nodes = 1
i = 0
i < num_downsample_connect = False
stride = 1
choice_keys = ['Reduce_n1_p0']
-------------------entrée dans la fonction __init__ de la classe PoolBN ------------------------
pool_type = max
C = 4
kernel_size = 3
stride = 1
padding = same
affine = False
-------------------entrée dans la fonction __init__ de la classe PoolBN ------------------------
pool_type = avg
C = 4
kernel_size = 3
stride = 1
padding = same
affine = False
self.ops = ListWrapper([LayerChoice(OrderedDict([('maxpool', <__main__.PoolBN object at 0x7f956afb1750>), ('avgpool', <__main__.PoolBN object at 0x7f94f4447450>), ('skipconnect', <tf.Tensor: shape=(), dtype=int32, numpy=4>), ('sepconv3x3', <__main__.SepConv object at 0x7f956afb1310>), ('sepconv5x5', <__main__.SepConv object at 0x7f9

/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: You should not run forward of this module directly.
  return py_builtins.overload_of(f)(*args)


--------entrée dans le call de la classe Node -------
prev_nodes = (<tf.Tensor 'IteratorGetNext:0' shape=(None, 32, 32, 4) dtype=float32>,)
len(prev_nodes) = 1
self.ops = ListWrapper([LayerChoice(OrderedDict([('maxpool', <__main__.PoolBN object at 0x7f956afb1750>), ('avgpool', <__main__.PoolBN object at 0x7f94f4447450>), ('skipconnect', <tf.Tensor: shape=(), dtype=int32, numpy=4>), ('sepconv3x3', <__main__.SepConv object at 0x7f956afb1310>), ('sepconv5x5', <__main__.SepConv object at 0x7f94f5e80090>), ('dilconv3x3', <__main__.DilConv object at 0x7f94f4eb9750>), ('dilconv5x5', <__main__.DilConv object at 0x7f956b3c00d0>)]), label='Reduce_n1_p0')])
len(self.ops) = 1
len(self.ops) == len(prev_nodes) = True
--------après assert--------
zip(self.ops, prev_nodes) = <zip object at 0x7f94f456f050>
-----------entrée dans la fonction call de PoolBN--------------
x = Tensor("IteratorGetNext:0", shape=(None, 32, 32, 4), dtype=float32)
self.pooling = <keras.layers.pooling.MaxPooling2D object at 0x7

--------entrée dans le call de la classe Node -------
prev_nodes = (<tf.Tensor 'IteratorGetNext:0' shape=(None, 32, 32, 4) dtype=float32>,)
len(prev_nodes) = 1
self.ops = ListWrapper([LayerChoice(OrderedDict([('maxpool', <__main__.PoolBN object at 0x7f956afb1750>), ('avgpool', <__main__.PoolBN object at 0x7f94f4447450>), ('skipconnect', <tf.Tensor: shape=(), dtype=int32, numpy=4>), ('sepconv3x3', <__main__.SepConv object at 0x7f956afb1310>), ('sepconv5x5', <__main__.SepConv object at 0x7f94f5e80090>), ('dilconv3x3', <__main__.DilConv object at 0x7f94f4eb9750>), ('dilconv5x5', <__main__.DilConv object at 0x7f956b3c00d0>)]), label='Reduce_n1_p0')])
len(self.ops) = 1
len(self.ops) == len(prev_nodes) = True
--------après assert--------
zip(self.ops, prev_nodes) = <zip object at 0x7f94f75777d0>
-----------entrée dans la fonction call de PoolBN--------------
x = Tensor("IteratorGetNext:0", shape=(None, 32, 32, 4), dtype=float32)
self.pooling = <keras.layers.pooling.MaxPooling2D object at 0x7

## classe Cell en tensorflow

### code à rajouter dans le fichier model.py

In [132]:
"""
import tensorflow as tf

class Cell(tf.keras.layers.Layer):
  def __init__(self, n_nodes, channels_pp, channels_p, channels, reduction_p, reduction):
    super().__init__()
    self.reduction = reduction
    self.n_nodes = n_nodes
    # If previous cell is reduction cell, current input size does not match with
    # output size of cell[k-2]. So the output[k-2] should be reduced by preprocessing.
    if reduction_p:
      self.preproc0 = ops.FactorizedReduce(channels_pp, channels, affine=False)
    else:
      self.preproc0 = ops.StdConv(channels_pp, channels, 1, 1, 0, affine=False)
    self.preproc1 = ops.StdConv(channels_p, channels, 1, 1, 0, affine=False)
    # generate dag
    self.mutable_ops = []
    for depth in range(2, self.n_nodes + 2):
      self.mutable_ops.append(Node("{}_n{}".format("reduce" if reduction else "normal", depth), 
                                   depth, channels, 2 if reduction else 0))
  def call(self, s0, s1):
    # s0, s1 are the outputs of previous previous cell and previous cell, respectively.
    tensors = [self.preproc0(s0), self.preproc1(s1)]
    for node in self.mutable_ops:
      cur_tensor = node(tensors)
      tensors.append(cur_tensor)
    output = output = tensors[2:]
    return output
"""

'\nimport tensorflow as tf\n\nclass Cell(tf.keras.layers.Layer):\n  def __init__(self, n_nodes, channels_pp, channels_p, channels, reduction_p, reduction):\n    super().__init__()\n    self.reduction = reduction\n    self.n_nodes = n_nodes\n    # If previous cell is reduction cell, current input size does not match with\n    # output size of cell[k-2]. So the output[k-2] should be reduced by preprocessing.\n    if reduction_p:\n      self.preproc0 = ops.FactorizedReduce(channels_pp, channels, affine=False)\n    else:\n      self.preproc0 = ops.StdConv(channels_pp, channels, 1, 1, 0, affine=False)\n    self.preproc1 = ops.StdConv(channels_p, channels, 1, 1, 0, affine=False)\n    # generate dag\n    self.mutable_ops = []\n    for depth in range(2, self.n_nodes + 2):\n      self.mutable_ops.append(Node("{}_n{}".format("reduce" if reduction else "normal", depth), \n                                   depth, channels, 2 if reduction else 0))\n  def call(self, s0, s1):\n    # s0, s1 are the o

### Code de test Cell

In [133]:
import tensorflow as tf

class Cell(tf.keras.layers.Layer):
  """
  Description
  ---------------
  This class allows the creation of the different cells of a neural network.
  """
  @Trace_appel()
  def __init__(self, n_nodes, channels_pp, channels_p, channels, reduction_p, reduction):
    """
    Description
    ---------------
    This function creates an instance of the Cell class.
    Input(s)
    ---------------
    n_nodes: int
    channels_pp: int
    channels_p: int
    channels: int
    reduction_p: int
    reduction: bool
    Output(s)
    ---------------
    no output
    """
    print("-----------------entrée dans la fonction __init__ de la classe Cell-------------------")
    super().__init__()
    self.reduction = reduction
    print("self.reduction = {}".format(self.reduction))
    self.n_nodes = n_nodes
    print("self.n_nodes = {}".format(self.n_nodes))
    # If previous cell is reduction cell, current input size does not match with
    # output size of cell[k-2]. So the output[k-2] should be reduced by preprocessing.
    if reduction_p:
      self.preproc0 = FactorizedReduce(channels_pp, channels, affine=False)
    else:
      self.preproc0 = StdConv(channels_pp, channels, 1, 1, "same", affine=False)
    self.preproc1 = StdConv(channels_p, channels, 1, 1, "same", affine=False)
    # generate dag
    self.mutable_ops = []
    for depth in range(2, self.n_nodes + 2):
      self.mutable_ops.append(Node("{}_n{}".format("reduce" if reduction else "normal", depth), 
                                   depth, channels, 2 if reduction else 0))

  #@Trace_appel()
  def call(self, s0, s1):
    """
    Description
    ---------------
    This function applies the instance of the Cell class to the output of
    the previous previous cell and to the output of the previous cell.
    Input(s)
    ---------------
    s0: tf.Tensor
    s1: tf.Tensor
    Output(s)
    ---------------
    output : tf.Tensor
    """
    print("----------------------entrée dans la fonction call de la classe Cell------------------------")
    # s0, s1 are the outputs of previous previous cell and previous cell, respectively.
    print("s0 = {}".format(s0))
    print("s1 = {}".format(s1))
    print("self.preproc0 = {}".format(self.preproc0))
    print("self.preproc0(s0) = {}".format(self.preproc0(s0)))
    tensors = [self.preproc0(s0), self.preproc1(s1)]
    print("self.mutable_ops = {}".format(self.mutable_ops))
    for node in self.mutable_ops:
      print("node = {}".format(node))
      print("tensors = {}".format(tensors))
      cur_tensor = node(tensors)
      print("cur_tensor = {}".format(cur_tensor))
      tensors.append(cur_tensor)
      print("tensors = {}".format(tensors))
    print("tensors[2:] = {}".format(tensors[2:]))
    output = tensors[2:]
    print("output = {}".format(output))
    return output


## Classe CNN en tensorflow

In [164]:
import tensorflow as tf
import tensorflow_lattice as tfl

class CNN(tf.keras.layers.Layer):
  """
   Description
   ---------------
   This class allows the creation of a CNN model.
  """
  def __init__(self, input_size, in_channels, channels, n_classes, n_layers, n_nodes=4,
               stem_multiplier=3, auxiliary=False):
    """
    Description
    ---------------
    This function creates an instance of the CNN class.
    Input(s)
    ---------------
    input_size: int
    in_channels: int
    channels: int
    n_classes: int
    n_layers: int
    n_nodes: int
    stem_multiplier: int
    auxiliary: bool
    Output(s)
    ---------------
    no output
    """
    print("---------------entrée dans la fonction init de la classe CNN---------------")
    super().__init__()
    self.in_channels = in_channels
    print("self.in_channels = {}".format(self.in_channels))
    self.channels = channels
    print("self.channels = {}".format(self.channels))
    self.n_classes = n_classes
    print("self.n_classes = {}".format(self.n_classes))
    print("-- avant la création de la variable self.n_layers --")
    self.n_layers = n_layers
    print("self.n_layers = {}".format(self.n_layers))
    print("auxiliary = {}".format(auxiliary))
    print("-- avant la création de la variable self.aux_pos --")
    self.aux_pos = 2 * n_layers // 3 if auxiliary else -1
    print("self.aux_pos = {}".format(self.aux_pos))
    print("-- avant la création de la variable c_cur --")
    print("stem_multiplier = {}".format(stem_multiplier))
    print("self.channels = {}".format(self.channels))
    c_cur = stem_multiplier * self.channels
    print("c_cur = {}".format(c_cur))
    print("-- avant la création de la layer self.conv2d --")
    self.conv2d = tf.keras.layers.Conv2D(filters=c_cur,
                                         strides=1,
                                         padding="same",
                                         data_format="channels_last",
                                         kernel_size=1,
                                         use_bias=False )
    print("self.conv2d = {}".format(self.conv2d))
    print("-- avant la création de la layer self.batch_normalisation --")
    self.batch_normalisation = tf.keras.layers.BatchNormalization(axis=-1)
    print("self.batch_normalisation = {}".format(self.batch_normalisation))
    # for the first cell, stem is used for both s0 and s1
    # [!] channels_pp and channels_p is output channel size, but c_cur is input channel size.
    print("-- avant l'assignement des variables channels_pp et channels_p --")
    print("c_cur = {}".format(c_cur))
    print("channels = {}".format(channels))
    channels_pp, channels_p, c_cur = c_cur, c_cur, channels
    print("-- après l'assignement des variables channels_pp, channels_p et c_cur --")
    print("channels_pp = {}".format(channels_pp))
    print("channels_p = {}".format(channels_p))
    print("c_cur = {}".format(c_cur))
    print("-- création de la liste self.cells --")
    self.cells = []
    print("self.cells")
    print("-- assignement des variables reduction_p et reduction --")
    reduction_p, reduction = False, False
    print("reduction_p = {}".format(reduction_p))
    print("reduction = {}".format(reduction))
    print("--avant l'entrée dans la boucle for --")
    for i in range(n_layers):
      print("-- dans la boucle for --")
      print("i = {}".format(i))
      print("-- avant l'assignement des variables reduction_p et reduction --")
      reduction_p, reduction = reduction, False
      print("reduction_p = {}".format(reduction_p))
      print("reduction = {}".format(reduction))
      # Reduce featuremap size and double channels in 1/3 and 2/3 layer.
      print("-- condition de la condition if --")
      print("n_layers // 3 = {}".format(n_layers // 3))
      print("2 * n_layers // 3 = {}".format(2 * n_layers // 3))
      print("i in [n_layers // 3, 2 * n_layers // 3] = {}".format(i in [n_layers // 3, 2 * n_layers // 3]))
      print("-- avant boucle if --")
      if i in [n_layers // 3, 2 * n_layers // 3]:
        print("-- dans la boucle if --")
        c_cur *= 2
        print("c_cur = {}".format(c_cur))
        reduction = True
        print("reduction = {}".format(reduction))
      print("-- avant la création de la cell --")
      print("paramètres:")
      print("n_nodes = {}".format(n_nodes))
      print("channels_pp = {}".format(channels_pp))
      print("channels_p = {}".format(channels_p))
      print("channels = {}".format(channels))
      print("reduction_p = {}".format(reduction_p))
      print("reduction = {}".format(reduction))
      cell = Cell(n_nodes, channels_pp, channels_p, channels, reduction_p, reduction)
      print("cell = {}".format(cell))
      self.cells.append(cell)
      print("self.cells = {}".format(self.cells))
      print("-- avant la création de c_cur_out --")
      print("c_cur = {}".format(c_cur))
      print("n_nodes = {}".format(n_nodes))
      c_cur_out = c_cur * n_nodes
      print("c_cur_out = {}".format(c_cur_out))
      print("-- avant l'assignement des variables channels_pp et channels_p --")
      print("channels_p = {}".format(channels_p))
      print("c_cur_out = {}".format(c_cur_out))
      channels_pp, channels_p = channels_p, c_cur_out
      print("-- après l'assignement des variables channels_pp et channels_p --")
      print("channels_pp = {}".format(channels_pp))
      print("channels_p = {}".format(channels_p))
      print("-- condition de la boucle if --")
      print("i = {}".format(i))
      print("self.aux_pos = {}".format(self.aux_pos))
      print("i == self.aux_pos = {}".format(i == self.aux_pos))
      if i == self.aux_pos:
        print("-- dans la boucle if --")
        print("paramètres:")
        print("input_size // 4 = {}".format(input_size // 4))
        print("channels_p = {}".format(channels_p))
        print("n_classes = {}".format(n_classes))
        self.aux_head = AuxiliaryHead(input_size // 4, channels_p, n_classes)
        print("self.aux_head = {}".format(self.aux_head))
    print("-- après la boucle for --")
    print("-- avant la creation de self.gap --")
    self.gap = tf.keras.layers.GlobalAveragePooling2D(keepdims=True)
    print("self.gap = {}".format(self.gap))
    print("-- avant la création de self.linear --")
    print("paramètre :")
    print("channels_p = {}".format(channels_p))
    print("n_classes = {}".format(n_classes))
    self.linear = tfl.layers.Linear(channels_p, n_classes)
    print("self.linear = {}".format(self.linear))

  
  def call(self, x):
    """
    Description
    ---------------
    This function applies the instance of the CNN class to the data.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    logits : tf.Tensor
    """
    print("-----------entrée dans le call de la classe CNN ------")
    out1 = self.conv2d(x)
    print("out1 = {}".format(out1))
    s0 = self.batch_normalisation(out1)
    s1 = self.batch_normalisation(out1)
    #s0 = s1 = self.stem(x)
    print("x = {}".format(x))
    print("s0 = {}".format(s0))
    print("s1 = {}".format(s1))
    #print("self.stem(x) = {}".format(self.stem(x)))
    aux_logits = None
    print("aux_logits = {}".format(aux_logits))
    print("self.cells = {}".format(self.cells))
    for i, cell in enumerate(self.cells):
      print("i = {}".format(i))
      print("cell = {}".format(cell))
      print("s0_avant_transformation = {}".format(s0))
      print("s1_avant_transformation = {}".format(s1))
      s0, s1 = s1, cell(s0, s1)
      print("s0_apres_transformation = {}".format(s0))
      print("s1_apres_transformation = {}".format(s1))
      print("self.aux_pos = {}".format(self.aux_pos))
      print("i == self.aux_pos = {}".format(i == self.aux_pos))
      print("i == self.aux_pos and self.training = {}".format(i == self.aux_pos and self.training))
      if i == self.aux_pos and self.training:
        print("self.aux_head = {}".format(self.aux_head))
        aux_logits = self.aux_head(s1)
        print("aux_logits = {}".format(aux_logits))
    print("self.gap = {}".format(self.gap))
    # ------
    if isinstance(s1,tf.Tensor):
      # s1 is a Tensor
      print("s1.get_shape() = {}".format(s1.get_shape()))
      out = self.gap(s1)
      print("out.get_shape() = {}".format(out.get_shape()))
    elif s1 == ():
      print("s1 is empty")
      out = ()
    else:
      # s1 is a list
      out = ()
      print("s1[0] = {}".format(s1[0]))
      print("len(s1) = {}".format(len(s1)))
      for i in range(len(s1)):
        print("i_cnn = {}".format(i))
        print("len(s1[i_cnn]) = {}".format(len(x[i])))
        if isinstance(len(s1[i]),int):
          # we want to know if x[i] is a tensor list
          if len(s1[i]) == 0 :
            # s1[i] contains no element
            print("s1[i_cnn] est vide")
          elif len(s1[i]) == 1 :
            # s1[i] contains only one tensor
            entree = s1[i][0]
            print("entree.get_shape() = {}".format(entree.get_shape()))
            out_i = self.gap(entree)
            print("out_i_cnn.get_shape() = {}".format(out_i.get_shape()))
            out = out + (out_i,)
          else:
            # s1[i] is a list containing more than one tensor
            print("s1[i] = {}".format(s1[i]))
            for j in range(len(s1[i])):
              entree = s1[i][j]
              print("entree = {}".format(entree))
              print("entree.get_shape() = {}".format(entree.get_shape()))
              print("entree.get_shape()[0] = {}".format(entree.get_shape()[0]))
              print("entree.get_shape()[1] = {}".format(entree.get_shape()[1]))
              print("entree.get_shape()[2] = {}".format(entree.get_shape()[2]))
              print("entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3 = {}".format(entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3))
              if entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3:
                # We check that the dimensions of the tensor respect the conditions necessary to apply the layer.
                out_i_j = self.gap(entree)
                print("out_i_cnn_j = {}".format(out_i_j))
                print("out_i_cnn_j.get_shape() = {}".format(out_i_j.get_shape()))
                out = out + (out_i_j,)
              else:
                out_i_j = tf.constant(1,shape=(3, 3, 3, 4))
                print("out_i_cnn_j = {}".format(out_i_j))
                print("out_i_cnn_j.get_shape() = {}".format(out_i_j.get_shape()))
                out = out + (out_i_j,)            
        else:
          # s1[i] is a Tensor
          out_i = self.gap(s1[i])
          print("out_i_cnn = {}".format(out_i))
          out = out + (out_i,)
    # -----
    print("out = {}".format(out))
    self.flatten = tf.keras.layers.Flatten()
    print("self.flatten = {}".format(self.flatten))
    # -----
    if isinstance(out,tf.Tensor):
      # out is a Tensor
      print("out_cnn.get_shape() = {}".format(out.get_shape()))
      out2 = self.flatten(out)
      print("out_cnn_apres_flatten.get_shape() = {}".format(out2.get_shape()))
    elif out == ():
      print("out is empty")
      out2 = ()
    else:
      # out is a list
      out2 = ()
      print("out[0] = {}".format(out[0]))
      print("len(out) = {}".format(len(out)))
      for i in range(len(out)):
        print("i_cnn_flatten = {}".format(i))
        print("len(out[i_cnn_flatten]) = {}".format(len(out[i])))
        if isinstance(len(out[i]),int):
          # we want to know if out[i] is a tensor list
          if len(out[i]) == 0 :
            # out[i] contains no element
            print("out[i_cnn_flatten] est vide")
          elif len(out[i]) == 1 :
            # out[i] contains only one tensor
            entree = out[i][0]
            print("entree.get_shape() = {}".format(entree.get_shape()))
            out_i = self.flatten(entree)
            print("out_i_cnn_flatten.get_shape() = {}".format(out_i.get_shape()))
            out2 = out2 + (out_i,)
          else:
            # out[i] is a list containing more than one tensor
            print("out[i_cnn_flatten] = {}".format(out[i]))
            for j in range(len(out[i])):
              entree = s1[i][j]
              print("entree = {}".format(entree))
              print("entree.get_shape() = {}".format(entree.get_shape()))
              print("entree.get_shape()[0] = {}".format(entree.get_shape()[0]))
              print("entree.get_shape()[1] = {}".format(entree.get_shape()[1]))
              print("entree.get_shape()[2] = {}".format(entree.get_shape()[2]))
              print("entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3 = {}".format(entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3))
              if entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3:
                # We check that the dimensions of the tensor respect the conditions necessary to apply the layer.
                out_i_j = self.flatten(entree)
                print("out_i_cnn_flatten_j = {}".format(out_i_j))
                print("out_i_cnn_flatten_j.get_shape() = {}".format(out_i_j.get_shape()))
                out2 = out2 + (out_i_j,)
              else:
                out_i_j = tf.constant(1,shape=(3, 3, 3, 4))
                print("out_i_cnn_j = {}".format(out_i_j))
                print("out_i_cnn_j.get_shape() = {}".format(out_i_j.get_shape()))
                out2 = out2 + (out_i_j,)    
        else:
          # out[i] is a Tensor
          out_i = self.flatten(out[i])
          print("out_i_cnn_flatten = {}".format(out_i))
          out2 = out2 + (out_i,)
    # ------
    #out = tf.keras.layers.Flatten()(out)
    #print("out.size(0) = {}".format(out.size(0)))
    #out = out.view(out.size(0), -1)  # flatten
    print("out2 = {}".format(out2))
    print("self.linear = {}".format(self.linear))
    # ------
    logits = []
    if isinstance(out2,tf.Tensor):
      # out is a Tensor
      print("out2_cnn.get_shape() = {}".format(out2.get_shape()))
      logits = self.linear(out2)
      print("logits_cnn.get_shape() = {}".format(logits.get_shape()))
    elif out2 == ():
      print("out2 is empty")
    else:
      # out2 is a list
      print("out2[0] = {}".format(out2[0]))
      print("len(out2) = {}".format(len(out2)))
      for i in range(len(out2)):
        print("i_cnn_linear = {}".format(i))
        print("len(out2[i_cnn_linear]) = {}".format(len(out2[i])))
        if isinstance(len(out2[i]),int):
          # we want to know if out2[i] is a tensor list
          if len(out2[i]) == 0 :
            # out2[i] contains no element
            print("out2[i_cnn_linear] est vide")
          elif len(out2[i]) == 1 :
            # out2[i] contains only one tensor
            entree = out2[i][0]
            print("entree.get_shape() = {}".format(entree.get_shape()))
            out_i = self.linear(entree)
            print("out_i_cnn_linear.get_shape() = {}".format(out_i.get_shape()))
            logits.append(out_i)
          else:
            # out2[i] is a list containing more than one tensor
            print("out2[i_cnn_linear] = {}".format(out2[i]))
            for j in range(len(out2[i])):
              entree = out2[i][j]
              print("entree = {}".format(entree))
              print("entree.get_shape() = {}".format(entree.get_shape()))
              print("entree.get_shape()[0] = {}".format(entree.get_shape()[0]))
              print("entree.get_shape()[1] = {}".format(entree.get_shape()[1]))
              print("entree.get_shape()[2] = {}".format(entree.get_shape()[2]))
              print("entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3 = {}".format(entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3))
              if entree.get_shape()[1] >= 3 and entree.get_shape()[2] >= 3:
                # We check that the dimensions of the tensor respect the conditions necessary to apply the layer.
                out_i_j = self.linear(entree)
                print("out_i_cnn_linear_j = {}".format(out_i_j))
                print("out_i_cnn_linear_j.get_shape() = {}".format(out_i_j.get_shape()))
                logits.append(out_i_j)
              else:
                out_i_j = tf.constant(1,shape=(3, 3, 3, 4))
                print("out_i_cnn_linear_j = {}".format(out_i_j))
                print("out_i_cnn_linear_j.get_shape() = {}".format(out_i_j.get_shape()))
                logits.append(out_i_j)    
        else:
          # out[i] is a Tensor
          print("logits = {}".format(logits))
          out_i = self.flatten(out2[i])
          print("out_i_cnn_linear = {}".format(out_i))
          logits.append(out_i)
    #logits = self.linear(out2)
    print("logits = {}".format(logits))
    print("aux_logits = {}".format(aux_logits))
    print("aux_logits is not None = {}".format(aux_logits is not None))
    # ------
    self.concatenation = tf.keras.layers.Concatenate(axis=-1)
    logits2 = self.concatenation(logits)
    print("logits2 = {}".format(logits2))
    if aux_logits is not None:
      return logits2, aux_logits
    return logits2

  def drop_path_prob(self, p):
    """
    Description
    ---------------
    This function drops some paths between neurons of the CNN model
    with a probability p.
    Input(s)
    ---------------
    p: float
    Output(s)
    ---------------
    no output
    """
    for module in self.modules():
      if isinstance(module, DropPath):
        module.p = p


## Test des classes du fichier model.py transformées en tensorflow

### Test de la classe CNN

In [166]:
import tensorflow as tf
print("=====================Test de la classe CNN ==============================")
nb_input = 10
print("nb_input = {}".format(nb_input))
test_tensors = tf.ones((nb_input,32,32,4))
print("isinstance(test_tensors, tf.Tensor) = {}".format(isinstance(test_tensors, tf.Tensor)))
print("test_tensors = {}".format(test_tensors))
test_model = tf.keras.models.Sequential()
test_model.add(CNN(4,4,4,4,4,4))
print("test_model = {}".format(test_model))
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,64)))
test_model.summary()

INFO:
INFO:>>>>>> Appel de __init__ >>>>>>
DEBUG:  ... Arguments :  N°1=<__main__.Cell object at 0x7f94f1ecfed0>   N°2=4   N°3=12   N°4=12   N°5=4   N°6=False   N°7=False 
INFO:
INFO:>>>>>> Appel de __init__ >>>>>>
DEBUG:  ... Arguments :  N°1=<__main__.Node object at 0x7f94f1834090>   N°2=normal_n2   N°3=2   N°4=4   N°5=0 


=====================Test de la classe CNN ==============================
nb_input = 10
isinstance(test_tensors, tf.Tensor) = True
test_tensors = [[[[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  ...

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]]


 [[[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   ...
   [1. 1. 

DEBUG:  ... Resultat : None
INFO:----- Fin de __init__ -----
INFO:
INFO:>>>>>> Appel de __init__ >>>>>>
DEBUG:  ... Arguments :  N°1=<__main__.Node object at 0x7f94f17c8ad0>   N°2=normal_n3   N°3=3   N°4=4   N°5=0 



n_candidates = 2
n_chosen = 2
label = normal_n2
self.input_switch = InputChoice(n_candidates=2, n_chosen=2, reduction='sum', label='normal_n2_switch')
----------------------entrée dans la fonction init de Node---------------------
num_prev_nodes = 3
i = 0
i < num_downsample_connect = False
stride = 1
choice_keys = ['normal_n3_p0']
-------------------entrée dans la fonction __init__ de la classe PoolBN ------------------------
pool_type = max
C = 4
kernel_size = 3
stride = 1
padding = same
affine = False
-------------------entrée dans la fonction __init__ de la classe PoolBN ------------------------
pool_type = avg
C = 4
kernel_size = 3
stride = 1
padding = same
affine = False
self.ops = ListWrapper([LayerChoice(OrderedDict([('maxpool', <__main__.PoolBN object at 0x7f94f46608d0>), ('avgpool', <__main__.PoolBN object at 0x7f94f4687a50>), ('skipconnect', <tf.Tensor: shape=(), dtype=int32, numpy=4>), ('sepconv3x3', <__main__.SepConv object at 0x7f94f4687dd0>), ('sepconv5x5', <__main__.Sep

DEBUG:  ... Resultat : None
INFO:----- Fin de __init__ -----
INFO:
INFO:>>>>>> Appel de __init__ >>>>>>
DEBUG:  ... Arguments :  N°1=<__main__.Node object at 0x7f94f1f216d0>   N°2=normal_n4   N°3=4   N°4=4   N°5=0 


...

self.ops = ListWrapper([LayerChoice(OrderedDict([('maxpool', <__main__.PoolBN object at 0x7f94ed018a10>), ('avgpool', <__main__.PoolBN object at 0x7f94ed01ec10>), ('skipconnect', <tf.Tensor: shape=(), dtype=int32, numpy=4>), ('sepconv3x3', <__main__.SepConv object at 0x7f94ed01ef10>), ('sepconv5x5', <__main__.SepConv object at 0x7f94ed038610>), ('dilconv3x3', <__main__.DilConv object at 0x7f94ecfcf490>), ('dilconv5x5', <__main__.DilConv object at 0x7f94ecfd5e10>)]), label='normal_n5_p0'), LayerChoice(OrderedDict([('maxpool', <__main__.PoolBN object at 0x7f94ecfdb410>), ('avgpool', <__main__.PoolBN object at 0x7f94ecfea410>), ('skipconnect', <tf.Tensor: shape=(), dtype=int32, numpy=4>), ('sepconv3x3', <__main__.SepConv object at 0x7f94ecfeac50>), ('sepconv5x5', <__main__.SepConv object at 0x7f94ecf87c50>), ('dilconv3x3', <__main__.DilConv object at 0x7f94ecfa2610>), ('dilconv5x5', <__main__.DilConv object at 0x7f94ecfaaf10>)]), label='normal_n5_p1'), LayerChoice(OrderedDict([('maxpool',

/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: You should not run forward of this module directly.
  return py_builtins.overload_of(f)(*args)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
-----------entrée dans la fonction call de PoolBN--------------
x = Tensor("cnn_17/cell_64/node_261/pool_bn_1804/batch_normalization_7632/FusedBatchNormV3:0", shape=(None, 32, 32, 4), dtype=float32)
self.pooling = <keras.layers.pooling.MaxPooling2D object at 0x7f94f70de950>
x.get_shape() = (None, 32, 32, 4)
out1 = Tensor("cnn_17/cell_64/node_263/pool_bn_1824/max_pooling2d_912/MaxPool:0", shape=(None, 32, 32, 4), dtype=float32)
out1.get_shape() = (None, 32, 32, 4)
out.get_shape() = (None, 32, 32, 4)
out = Tensor("cnn_17/cell_64/node_263/pool_bn_1824/batch_normalization_7712/FusedBatchNormV3:0", shape=(None, 32, 32, 4), dtype=float32)
-----------entrée dans la fonction call de PoolBN--------------
x = Tensor("cnn_17/cell_64/node_262/pool_bn_1810/batch_normalization_7656/FusedBatchNormV3:0", shape=(None, 32, 32, 4), dtype=float32)
self.pooling = <keras.layers.pooling.MaxPooling2D object at 0x7f94ed82aa10>
x.get_

-----------entrée dans le call de la classe CNN ------
out1 = Tensor("sequential_7242/cnn_17/conv2d_11510/Conv2D:0", shape=(None, 32, 32, 12), dtype=float32)
x = Tensor("IteratorGetNext:0", shape=(None, 32, 32, 4), dtype=float32)
s0 = Tensor("sequential_7242/cnn_17/batch_normalization_7613/FusedBatchNormV3:0", shape=(None, 32, 32, 12), dtype=float32)
s1 = Tensor("sequential_7242/cnn_17/batch_normalization_7613/FusedBatchNormV3_1:0", shape=(None, 32, 32, 12), dtype=float32)
aux_logits = None
self.cells = ListWrapper([<__main__.Cell object at 0x7f94f1ecfed0>, <__main__.Cell object at 0x7f94f30d2950>, <__main__.Cell object at 0x7f94efbbbb50>, <__main__.Cell object at 0x7f94f23259d0>])
i = 0
cell = <__main__.Cell object at 0x7f94f1ecfed0>
s0_avant_transformation = Tensor("sequential_7242/cnn_17/batch_normalization_7613/FusedBatchNormV3:0", shape=(None, 32, 32, 12), dtype=float32)
s1_avant_transformation = Tensor("sequential_7242/cnn_17/batch_normalization_7613/FusedBatchNormV3_1:0", shape=

1/1 [==============================] - 29s 29s/step - loss: 1.0000
Model: "sequential_7242"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_17 (CNN)                (None, 64)                19232     
                                                                 
Total params: 19,232
Trainable params: 18,696
Non-trainable params: 536
_________________________________________________________________


# Compréhension du code initial



from collections import OrderedDict

import torch
import torch.nn as nn

import ops
from nni.retiarii.nn.pytorch import LayerChoice, InputChoice


class AuxiliaryHead(nn.Module):
    """ Auxiliary head in 2/3 place of network to let the gradient flow well """

    def __init__(self, input_size, C, n_classes):
        """ assuming input size 7x7 or 8x8 """
        assert input_size in [7, 8]
        # on vérifie notre hypothèse initiale qui est que les données ont une taille égale à 7x7 ou à 8x8
        super().__init__()
        # on fait en sorte que AuxiliaryHead soit une cellule qu'il est possible de mettre dans un modèle de type Sequential 
        self.net = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.AvgPool2d(5, stride=input_size - 5, padding=0, count_include_pad=False),  # 2x2 out
            nn.Conv2d(C, 128, kernel_size=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 768, kernel_size=2, bias=False),  # 1x1 out
            nn.BatchNorm2d(768),
            nn.ReLU(inplace=True)
        )
        # on crée notre modèle correspondant à notre tête de réseau de neurone : cette tête est composée de plusieurs opérations une relu, un average pooling de dimension 5, une convolution, une batchnormalisation, une relu, une convolution, une batchnormalisation et une relu. 
        self.linear = nn.Linear(768, n_classes)
        # on met ensuite une cellule linéaire pour pouvoir obtenir une classification du réseau sur n_classes.

    def forward(self, x):
        # cette fonction permet d'appliquer la tête du modèle à un tenseur (ce qui correspond aux données d'entrée du modèle que l'on veut construire)
        out = self.net(x)
        # on applique le modèle créé précédemment dans la fonction init aux données d'entrée (qui sont un tenseur).
        out = out.view(out.size(0), -1)  # flatten
        # on effectue un flaten, ce qui permet d'obtenir un vecteur en sortie de notre tête du modèle.
        logits = self.linear(out)
        # on applique la fonction linéaire pour ???
        return logits

class Node(nn.Module):
    # classe qui permet de créer les noeuds du modèle
    def __init__(self, node_id, num_prev_nodes, channels, num_downsample_connect):
        # on initialise les noeuds (i.e on crée les noeuds du modèle).
        super().__init__()
        # on fait en sorte que les noeuds puissent être considéré comme étant des layers que l'on peut rajouter au modèle.
        self.ops = nn.ModuleList()
        # on crée une liste afin de pouvoir par la suite lister les opérations que l'on veut tester dans le cadre de la construction du modèle à l'aide du DARTS
        choice_keys = []
        # on crée une liste afin de pouvoir tester les clefs ???
        for i in range(num_prev_nodes):
            # on parcourt les noeuds déjà créés auparavant.
            stride = 2 if i < num_downsample_connect else 1
            # on regarde si l'on désire diminuer la taille des tenseurs résultant des différentes opérations créés par le Darts pour le modèle
            choice_keys.append("{}_p{}".format(node_id, i))
            # on rajoute les clefs dans la liste des clefs choisies
            self.ops.append(
                LayerChoice(OrderedDict([
                  # on veut choisir une layer parmi les suivantes dans un dictionnaire que l'on crée.
                    ("maxpool", ops.PoolBN('max', channels, 3, stride, 1, affine=False)),
                    # le premier élément du dictionnaire est une opération de maxpooling.
                    ("avgpool", ops.PoolBN('avg', channels, 3, stride, 1, affine=False)),
                    # la seconde opération que l'on peut choisir est une opération d'average pooling. 
                    ("skipconnect", nn.Identity() if stride == 1 else ops.FactorizedReduce(channels, channels, affine=False)),
                    # la troisième opération correspond à une opération de skipconnect i.e. une opération qui consiste en le saut de plusieurs liens entre neurones afin de favoriser la backpropagation du gradient et donc l'entraînement du modèle.
                    ("sepconv3x3", ops.SepConv(channels, channels, 3, stride, 1, affine=False)),
                    # la quatrième opération est une convolution séparée qui consiste à ??? . elle est de taille 3x3.
                    ("sepconv5x5", ops.SepConv(channels, channels, 5, stride, 2, affine=False)),
                    # la cinquième opération correspond à une convolution de taille 5x5.
                    ("dilconv3x3", ops.DilConv(channels, channels, 3, stride, 2, 2, affine=False)),
                    # la sixième opération est une convolution dilatée de taille 3x3. La convolution dilatée consiste en ??? .
                    ("dilconv5x5", ops.DilConv(channels, channels, 5, stride, 4, 2, affine=False))
                    # Enfin, la septième opération consiste en la même chose que la sixième opération sauf que la taille de la convolution est désormais 5x5 .
                ]), label=choice_keys[-1]))
                # le label correspond aux clef qui peuvent être utilisées afin de pouvoir choisir les opérations que l'on veut utiliser afin de créer le modèle à l'aide du DARTS.
        self.drop_path = ops.DropPath()
        # l'opération de drop path permet de retirer du schéma du modèle créé par le DARTS les chemins qui ne sont pas intéressants.
        self.input_switch = InputChoice(n_candidates=len(choice_keys), n_chosen=2, label="{}_switch".format(node_id))
        # La fonction Input Choice permet de choisir les noeuds que l'on veut conserver dans le modèle créé par le DARTS.

    def forward(self, prev_nodes):
        # on applique à présent les cellules sur les neurones qui ont déjà été créés précédemment.
        assert len(self.ops) == len(prev_nodes)
        # on vérifie qu'il y a autant d'opérations que de noeuds.
        out = [op(node) for op, node in zip(self.ops, prev_nodes)]
        # on applique les opérations à chaque noeuds afin de tester toutes les possibilitées lors de la construction du modèle par le DARTS.
        out = [self.drop_path(o) if o is not None else None for o in out]
        # on enlève les chemins entre les noeuds qui ne sont pas dans la liste des opérations appliquées à chaque noeuds que l'on a créée précédemment.
        return self.input_switch(out)


class Cell(nn.Module):
    # cette classe permet de créer les cellules du modèle dont on veut optimiser l'architecture lors de sa création à l'aide du DARTS.
    def __init__(self, n_nodes, channels_pp, channels_p, channels, reduction_p, reduction):
        super().__init__()
        # on fait en sorte que les cellules soient des couches afin de pouvoir les rajouter au modèle séquential.
        self.reduction = reduction
        # on met l'argument reduction dans une variable self.reduction afin de pouvoir savoir durant toute la création du modèle à l'aide du DARTS si une cellule est une cellule dite de réduction ou si la cellule est une cellule normale.
        self.n_nodes = n_nodes
        # on met l'argument n_nodes dans une variable self.n_nodes afin de pouvoir savoir combien de node on désire avoir dans la cellule que l'on est en train de créer.

        # If previous cell is reduction cell, current input size does not match with
        # output size of cell[k-2]. So the output[k-2] should be reduced by preprocessing.
        if reduction_p:
            # on regarde si la cellule précédente est une cellule de réduction ou pas.
            # en effet, si tel est le cas, alors la taille d'entrée de la cellule que l'on est en train d'initialiser n'est pas la bonne.
            self.preproc0 = ops.FactorizedReduce(channels_pp, channels, affine=False)
            # comme c'est le cas dans cette partie du code, on applique une méthode de préprocessing afin de pouvoir transformer la taille du tenseur d'entrée et avoir ainsi un tenseur en entrée de la cellule qui ait la bonne taille.
        else:
            # ici on est dans le cas où la cellule précédente n'est pas une cellule de réduction.
            self.preproc0 = ops.StdConv(channels_pp, channels, 1, 1, 0, affine=False)
            # on effectue cela pour ???
        self.preproc1 = ops.StdConv(channels_p, channels, 1, 1, 0, affine=False)
        # on effectue cela pour ???

        # generate dag
        self.mutable_ops = nn.ModuleList()
        # on crée une liste afin de stocker les opérations que l'on peut effectuer.
        for depth in range(2, self.n_nodes + 2):
            # ensuite on parcourt la liste des n_nodes précédents
            self.mutable_ops.append(Node("{}_n{}".format("reduce" if reduction else "normal", depth),
                                         depth, channels, 2 if reduction else 0))
            # ensuite, on rajoute dans la liste des opérations appliquées aux noeuds un nouveau noeud que l'on crée avec une opération de type reduce si l'on a la condition reduction == True, sinon le noeud a une opération normale.

    def forward(self, s0, s1):
        # on applique la cellule aux deux outputs des cellules précédentes i.e. à l'output de la cellule crée juste avant la cellule précédente et à l'output de la cellule précédente.
        # s0, s1 are the outputs of previous previous cell and previous cell, respectively.
        tensors = [self.preproc0(s0), self.preproc1(s1)]
        # on met les 2 tenseurs résultant des deux cellules précédentes dans une liste que l'on appelle "tensors". 
        for node in self.mutable_ops:
            # on regarde les noeuds qui sont situés dans la liste des opérations appliquées aux noeuds.
            cur_tensor = node(tensors)
            # on appelle la fonction call de la classe Node car on instancie le noeud choisi en l'appliquant à la liste composé des 2 tenseurs.
            tensors.append(cur_tensor)
            # on rajoute le tenseur créé dans la liste des 2 tenseurs.

        output = torch.cat(tensors[2:], dim=1)
        # ceci permet de ??? --> peut-être que cela permet uniquement de ne choisir que les 2 derniers tenseurs afin que l'on soit toujours avec le tenseur de la cellule avant la cellule précédente et avec le tenseur de la cellule précédente.
        return output
        # ceci permet de retourner ce que l'on désire i.e. les 2 nouveaux tenseurs dont on a besoin pour la cellule suivante.


class CNN(nn.Module):
    # Ceci permet au DARTS de créer le modèle CNN le plus adapté en améliorant sa structure.
    def __init__(self, input_size, in_channels, channels, n_classes, n_layers, n_nodes=4,
                 stem_multiplier=3, auxiliary=False):
        # ce code permet d'initialiser le CNN que l'on veut créer et dont on veut optimiser l'architecture à l'aide du DARTS.
        super().__init__()
        # on se débrouille pour que le CNN soit une layer que l'on peut mettre dans un modèle sequential afin de pouvoir créer cette cellule à l'aide de la méthode DARTS.
        self.in_channels = in_channels
        # on prend le paramètre in_channels pour que l'on puisse avoir le nombre de channels d'entrée et pour que l'on puisse mettre à jour le nombre de channels de chaque cellule que l'on va utiliser lors de la création du modèle CNN à l'aide du DARTS.
        self.channels = channels
        # ce paramètres correspond aux nombre de channels que l'on désire avoir à l'intérieur des différentes cellules que l'on utilise lors de la création du modèle CNN par le DARTS.
        self.n_classes = n_classes
        # ceci correspond au nombre de classe que l'on désire avoir en sortie de notre modèle CNN généré par la méthode DARTS.
        self.n_layers = n_layers
        # ceci correspond au nombre de couches ("layers") que l'on désire avoir dans le modèle CNN que l'on désire créer à l'aide de la méthode DARTS. 
        self.aux_pos = 2 * n_layers // 3 if auxiliary else -1
        # ceci permet de ???
        c_cur = stem_multiplier * self.channels
        # ceci permet de ???
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, c_cur, 3, 1, 1, bias=False),
            nn.BatchNorm2d(c_cur)
        )
        # on crée le modèle CNN comme étant un modèle séquentiel. Il est composé de plusieurs éléments 

        # for the first cell, stem is used for both s0 and s1
        # [!] channels_pp and channels_p is output channel size, but c_cur is input channel size.
        channels_pp, channels_p, c_cur = c_cur, c_cur, channels
        # on met à jour la variable c_cur
        # et l'on change les valeurs des variables channels_pp et channels_p.
        self.cells = nn.ModuleList()
        # on crée une liste des cellules que l'on va utiliser pour pouvoir créer notre modèle CNN à l'aide de la méthode DARTS.
        reduction_p, reduction = False, False
        # on crée les variables reduction_p et reduction en les initialisant à la valeur "False".
        for i in range(n_layers):
            # on parcourt les indices des couches (layers) du modèle CNN 
            reduction_p, reduction = reduction, False
            # on met à jour les variables reduction_p et reduction afin de mettre la valeur de la variable reduction à False et de mettre la valeur de la variable reduction_p à la valeur précédente de la variable reduction.
            # Reduce featuremap size and double channels in 1/3 and 2/3 layer.
            if i in [n_layers // 3, 2 * n_layers // 3]:
                # on parcourt à l'aide de l'indice i la taille de la map des features que l'on désire réduire d'un tiers et le nombre de channels que l'on veut doubler.
                c_cur *= 2
                # ceci permet de ???
                reduction = True
                # on met la valeur de la variable reduction à True car notre algorithme DARTS nécessite de créer des noeuds avec des opérations de réduction.

            cell = Cell(n_nodes, channels_pp, channels_p, c_cur, reduction_p, reduction)
            # On crée une cellule contenant un nombre de noeuds égal à n_nodes. 
            self.cells.append(cell)
            # on rajoute ensuite la cellule créée dans la liste de cellule self.cells .
            c_cur_out = c_cur * n_nodes
            # ceci permet de ???
            channels_pp, channels_p = channels_p, c_cur_out
            # on met à jour les variables channels_pp et channels_p en mettant la variable channels_pp qui correspond au nombre de channels dans la couche ("layer") créé juste avant la couche ("layer") précédente à jour : quand on a fini de créer la couche sur laquelle l'algorithme DARTS était en train de travailler, le nombre de channel de la couche précédant celle que l'on était en train de créer devient le nouveau nombre de couche de la couche précédant la couche précédente et lorsque l'on crée une nouvelle couche, la couche que l'on a terminé de créer devient tout simplement la couche précédant la nouvelle couche.
            if i == self.aux_pos:
                # on vérifie si i a atteint la fin 
                self.aux_head = AuxiliaryHead(input_size // 4, channels_p, n_classes)
                # et on ajoute une tête auxiliaire au modèle afin de pouvoir faciliter la backpropagation du gradient, ce qui est nécessaire dans le cadre de l'entraînement du modèle 
        self.gap = nn.AdaptiveAvgPool2d(1)
        # on crée une layer correspondant à un average pooling qui est adaptatif, i.e. qui effectue un average pooling avec une moyenne glissante.
        self.linear = nn.Linear(channels_p, n_classes)
        # Enfin, on crée une layer fonctionnelle qui est linéaire.

    def forward(self, x):
        # on applique le modèle CNN créé par le DARTS à des données qui sont représentées par le tenseur x
        s0 = s1 = self.stem(x)
        # on applique le modèle stem de base du CNN afin de pouvoir effectuer un préprocessing des données
        aux_logits = None
        # au tout début, il n'y a pas de logits donc la variable aux_logits est initialisée à None
        for i, cell in enumerate(self.cells):
            # on énumère l'indice des cellules ainsi que les cellules initialisées précédemment afin de pouvoir effectivement créer les poids des cellules.
            s0, s1 = s1, cell(s0, s1)
            # ensuite, on crée effectivement les poids des cellules en les appliquants sur les données s0 et s1 (qui sont des tenseurs). 
            if i == self.aux_pos and self.training:
                # on regarde si l'indice de la cellule considéré est le bon indice et si l'on est en train d'entraîner le modèle CNN.
                aux_logits = self.aux_head(s1)
                # on met à jour la variable aux_logits pour ???
        out = self.gap(s1)
        # on appplique ensuite le modèle self.gap qui correspond à une layer de average pooling avec une moyenne glissante sur le tenseur s1
        out = out.view(out.size(0), -1)  # flatten
        # Ensuite on effectue un flatten afin de pouvoir obtenir un vecteur résultat.
        logits = self.linear(out)
        # ceci permet de ???

        if aux_logits is not None:
        # ceci permet de ???
            return logits, aux_logits
            # ceci permet d'obtenir les logits et les logits auxiliaires.
        return logits
        # ceci permet d'obtenir les logits car il n'y a pas de logits auxiliaires.

    def drop_path_prob(self, p):
        # ceci permet de retirer le chemin des neurones avec une probabilité p
        for module in self.modules():
            # on regarde les modules de la liste des modules.
            if isinstance(module, ops.DropPath):
            # on regarde si le module est une instance des opérations du chemin
                module.p = p
                # et on met la probabilité de retirer le module à p
